<a href="https://colab.research.google.com/github/JasonTJH/mycolab/blob/main/TopNewFollowers_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Social Bakers (Top Twitter New Followers)

In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import time
import sys
import re
from bs4 import BeautifulSoup, SoupStrainer
from urllib.parse import urljoin
from selenium import webdriver
import subprocess
from itertools import repeat, chain
from multiprocessing import Pool, Process, Manager
import requests
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed
from concurrent.futures import Future
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from collections import OrderedDict

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

     |████████████████████████████████| 911kB 3.9MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted 

## Setup

In [3]:
# Defining static & test variables
base = 'https://www.socialbakers.com'
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get(base+'/statistics/twitter/profiles')
soup = BeautifulSoup(driver.page_source, "html.parser")
sample_link_ext = '/statistics/twitter/profiles/afghanistan'
sample_profile_ext = '/statistics/twitter/profiles/detail/111240404-radiomosaiquefm'
sample_chart_link = "https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids=111240404&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months"
industry_list = ['/statistics/twitter/profiles', '/statistics/twitter/profiles/brands', '/statistics/twitter/profiles/celebrities', '/statistics/twitter/profiles/community', '/statistics/twitter/profiles/entertainment', '/statistics/twitter/profiles/media', '/statistics/twitter/profiles/place', '/statistics/twitter/profiles/society', '/statistics/twitter/profiles/sport']
sleep_time = 0.05 #for loading javascript in selenium parser - raise it if get_followers have no output

# Functions to be used below
def get_profile_ext(link_ext):
  my_list = list()
  driver.get(base + link_ext)
  soup = BeautifulSoup(driver.page_source, "lxml", parse_only=SoupStrainer(href=re.compile("/statistics/twitter/profiles/detail/")))
  outputs = [str(i) for i in soup]
  splitter = 'href="'
  splitter2 = '-'
  splitter3 = '"'
  for a in outputs:
    my_list.append(a.split(splitter)[1].split(splitter2)[0].split(splitter3)[0])
  return my_list
#print(get_profile_ext(sample_link_ext))


def get_chart_link(profile_ext):
  chart_link = 'https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids='+profile_ext.split('/detail/')[1].split('-')[0]+'&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months'
  return chart_link
#print(get_chart_link(sample_profile_ext))

def get_followers(chart_link):
  #driver = webdriver.Chrome('chromedriver',options=chrome_options)
  driver.get(chart_link)
  try:
    time.sleep(sleep_time)
    result = driver.execute_script('var text = options ; return text')
  except:
    try:
      time.sleep(sleep_time)
      result = driver.execute_script('var text = options ; return text')
    except:
      try:
        time.sleep(sleep_time)
        result = driver.execute_script('var text = options ; return text')
      except:
        return [], [], []
  # Cleaning data for use
  today = datetime(datetime.now().year, datetime.now().month, datetime.now().day)

  follower_list = []
  num_list = list(range(14*12,-14,-14))
  date_list = [today - timedelta(days= i) for i in num_list]
  for i,v in dict(dict(result)['series'][0])['data']:
    follower_list.append(v)
  return date_list, follower_list, num_list
#print(get_followers(sample_chart_link))

def plot_g(date_list, follower_list):
  plt.plot(date_list, follower_list)
#x, y, z = get_followers(sample_chart_link)
#plot_g(x,y)

def pct_chg(follower_list, num_list):
  try:
    followers_bf_2w = follower_list[num_list.index(14)] + 1 if follower_list[num_list.index(14)] == 0 else follower_list[num_list.index(14)]
    followers_bf_1m = follower_list[num_list.index(28)] + 1 if follower_list[num_list.index(28)] == 0 else follower_list[num_list.index(28)]
    followers_bf_3m = follower_list[num_list.index(84)] + 1 if follower_list[num_list.index(84)] == 0 else follower_list[num_list.index(84)]
    followers_bf_6m = follower_list[num_list.index(168)] + 1 if follower_list[num_list.index(168)] == 0 else follower_list[num_list.index(168)]
    followers_af = follower_list[-1]
    pct_chg_2w = round((followers_af/followers_bf_2w-1),5)
    pct_chg_1m = round((followers_af/followers_bf_1m-1),5)
    pct_chg_3m = round((followers_af/followers_bf_3m-1),5)
    pct_chg_6m = round((followers_af/followers_bf_6m-1),5)
    followers_bf_list = [followers_bf_6m, followers_bf_3m, followers_bf_1m, followers_bf_2w]
    pct_chg_list = [pct_chg_6m,pct_chg_3m,pct_chg_1m,pct_chg_2w]
  except:
    return
  return pct_chg_list, followers_bf_list, followers_af

#x, y, z = get_followers(sample_chart_link)
#print(pct_chg(y,z))

def get_category(profile_ext):
  #profile_ext = chart_link.split('?ids=')[1].split('&colors=')[0]
  driver.get(base + profile_ext)
  soup = BeautifulSoup(driver.page_source, "html.parser")
  category_set = set()
  for a in soup.find_all(href=True):
    if a['href'].startswith('/statistics/twitter/profiles/') and '/statistics/twitter/profiles/detail' not in a['href'] and a['href'] not in industry_list:
      category_set.add(str(a['href']).split('profiles/')[1])
  return category_set
#print(get_category(sample_profile_ext))

def get_profile_name(profile_ext):
  driver.get(base + profile_ext)
  soup = BeautifulSoup(driver.page_source, "html.parser")
  category_set = set()
  for a in soup.find_all('a', {'class' : 'external link-nw'}):
    profile_name = a.text
  return profile_name
#print(get_profile_name(sample_profile_ext))

def set_var(tenor,top_x):
  if tenor == '6m':
    target_list = sorted_chg_6m[len(sorted_chg_6m)-top_x:]
    target_value = 0
  elif tenor == '3m':
    target_list = sorted_chg_3m[len(sorted_chg_3m)-top_x:]
    target_value = 1
  elif tenor == '1m':
    target_list = sorted_chg_1m[len(sorted_chg_1m)-top_x:]
    target_value = 2
  elif tenor == '2w':
    target_list = sorted_chg_2w[len(sorted_chg_2w)-top_x:]
    target_value = 3
  else:
    pass
  return target_list, target_value, tenor, top_x
#print(set_var('2w',25)) # var1 takes in ['2w','1m','3m','6m'], var2 is the top # results you want to see - requires some output from code below before testing is allowed

### Getting Active List

In [7]:
# Getting a list of directories to each countries page
active_list = ['/statistics/twitter/profiles/afghanistan', '/statistics/twitter/profiles/aland-islands', '/statistics/twitter/profiles/albania', '/statistics/twitter/profiles/algeria', '/statistics/twitter/profiles/american-samoa', '/statistics/twitter/profiles/andorra', '/statistics/twitter/profiles/angola', '/statistics/twitter/profiles/anguilla', '/statistics/twitter/profiles/antigua-and-barbuda', '/statistics/twitter/profiles/argentina', '/statistics/twitter/profiles/armenia', '/statistics/twitter/profiles/aruba', '/statistics/twitter/profiles/australia', '/statistics/twitter/profiles/austria', '/statistics/twitter/profiles/azerbaijan', '/statistics/twitter/profiles/bahamas', '/statistics/twitter/profiles/bahrain', '/statistics/twitter/profiles/bangladesh', '/statistics/twitter/profiles/barbados', '/statistics/twitter/profiles/belarus', '/statistics/twitter/profiles/belgium', '/statistics/twitter/profiles/belize', '/statistics/twitter/profiles/benin', '/statistics/twitter/profiles/bermuda', '/statistics/twitter/profiles/bhutan', '/statistics/twitter/profiles/bolivia', '/statistics/twitter/profiles/bonaire-saint-eustatius-and-saba', '/statistics/twitter/profiles/bosnia-and-herzegovina', '/statistics/twitter/profiles/botswana', '/statistics/twitter/profiles/brazil', '/statistics/twitter/profiles/british-indian-ocean-territory', '/statistics/twitter/profiles/british-virgin-islands', '/statistics/twitter/profiles/brunei', '/statistics/twitter/profiles/bulgaria', '/statistics/twitter/profiles/burkina-faso', '/statistics/twitter/profiles/burundi', '/statistics/twitter/profiles/cambodia', '/statistics/twitter/profiles/cameroon', '/statistics/twitter/profiles/canada', '/statistics/twitter/profiles/cape-verde', '/statistics/twitter/profiles/cayman-islands', '/statistics/twitter/profiles/central-african-republic', '/statistics/twitter/profiles/chad', '/statistics/twitter/profiles/chile', '/statistics/twitter/profiles/china', '/statistics/twitter/profiles/cocos-islands', '/statistics/twitter/profiles/colombia', '/statistics/twitter/profiles/comoros', '/statistics/twitter/profiles/costa-rica', '/statistics/twitter/profiles/croatia', '/statistics/twitter/profiles/cuba', '/statistics/twitter/profiles/curacao', '/statistics/twitter/profiles/cyprus', '/statistics/twitter/profiles/czech-republic', '/statistics/twitter/profiles/democratic-republic-of-the-congo', '/statistics/twitter/profiles/denmark', '/statistics/twitter/profiles/djibouti', '/statistics/twitter/profiles/dominica', '/statistics/twitter/profiles/dominican-republic', '/statistics/twitter/profiles/east-timor', '/statistics/twitter/profiles/ecuador', '/statistics/twitter/profiles/egypt', '/statistics/twitter/profiles/el-salvador', '/statistics/twitter/profiles/equatorial-guinea', '/statistics/twitter/profiles/eritrea', '/statistics/twitter/profiles/estonia', '/statistics/twitter/profiles/ethiopia', '/statistics/twitter/profiles/falkland-islands', '/statistics/twitter/profiles/faroe-islands', '/statistics/twitter/profiles/fiji', '/statistics/twitter/profiles/finland', '/statistics/twitter/profiles/france', '/statistics/twitter/profiles/french-guiana', '/statistics/twitter/profiles/french-polynesia', '/statistics/twitter/profiles/global', '/statistics/twitter/profiles/gabon', '/statistics/twitter/profiles/gambia', '/statistics/twitter/profiles/georgia', '/statistics/twitter/profiles/germany', '/statistics/twitter/profiles/ghana', '/statistics/twitter/profiles/gibraltar', '/statistics/twitter/profiles/greece', '/statistics/twitter/profiles/greenland', '/statistics/twitter/profiles/grenada', '/statistics/twitter/profiles/guadeloupe', '/statistics/twitter/profiles/guam', '/statistics/twitter/profiles/guatemala', '/statistics/twitter/profiles/guernsey', '/statistics/twitter/profiles/guinea', '/statistics/twitter/profiles/guinea-bissau', '/statistics/twitter/profiles/guyana', '/statistics/twitter/profiles/haiti', '/statistics/twitter/profiles/honduras', '/statistics/twitter/profiles/hong-kong', '/statistics/twitter/profiles/hungary', '/statistics/twitter/profiles/iceland', '/statistics/twitter/profiles/india', '/statistics/twitter/profiles/indonesia', '/statistics/twitter/profiles/iran', '/statistics/twitter/profiles/iraq', '/statistics/twitter/profiles/ireland', '/statistics/twitter/profiles/isle-of-man', '/statistics/twitter/profiles/israel', '/statistics/twitter/profiles/italy', '/statistics/twitter/profiles/ivory-coast', '/statistics/twitter/profiles/jamaica', '/statistics/twitter/profiles/japan', '/statistics/twitter/profiles/jersey', '/statistics/twitter/profiles/jordan', '/statistics/twitter/profiles/kazakhstan', '/statistics/twitter/profiles/kenya', '/statistics/twitter/profiles/kosovo', '/statistics/twitter/profiles/kuwait', '/statistics/twitter/profiles/kyrgyzstan', '/statistics/twitter/profiles/laos', '/statistics/twitter/profiles/latvia', '/statistics/twitter/profiles/lebanon', '/statistics/twitter/profiles/lesotho', '/statistics/twitter/profiles/liberia', '/statistics/twitter/profiles/libya', '/statistics/twitter/profiles/liechtenstein', '/statistics/twitter/profiles/lithuania', '/statistics/twitter/profiles/luxembourg', '/statistics/twitter/profiles/macao', '/statistics/twitter/profiles/macedonia', '/statistics/twitter/profiles/madagascar', '/statistics/twitter/profiles/malawi', '/statistics/twitter/profiles/malaysia', '/statistics/twitter/profiles/maldives', '/statistics/twitter/profiles/mali', '/statistics/twitter/profiles/malta', '/statistics/twitter/profiles/marshall-islands', '/statistics/twitter/profiles/martinique', '/statistics/twitter/profiles/mauritania', '/statistics/twitter/profiles/mauritius', '/statistics/twitter/profiles/mayotte', '/statistics/twitter/profiles/mexico', '/statistics/twitter/profiles/micronesia', '/statistics/twitter/profiles/moldova', '/statistics/twitter/profiles/monaco', '/statistics/twitter/profiles/mongolia', '/statistics/twitter/profiles/montenegro', '/statistics/twitter/profiles/montserrat', '/statistics/twitter/profiles/morocco', '/statistics/twitter/profiles/mozambique', '/statistics/twitter/profiles/myanmar', '/statistics/twitter/profiles/namibia', '/statistics/twitter/profiles/nauru', '/statistics/twitter/profiles/nepal', '/statistics/twitter/profiles/netherlands', '/statistics/twitter/profiles/netherlands-antilles', '/statistics/twitter/profiles/new-caledonia', '/statistics/twitter/profiles/new-zealand', '/statistics/twitter/profiles/nicaragua', '/statistics/twitter/profiles/niger', '/statistics/twitter/profiles/nigeria', '/statistics/twitter/profiles/norfolk-island', '/statistics/twitter/profiles/northern-mariana-islands', '/statistics/twitter/profiles/norway', '/statistics/twitter/profiles/oman', '/statistics/twitter/profiles/pakistan', '/statistics/twitter/profiles/palau', '/statistics/twitter/profiles/palestinian-territory', '/statistics/twitter/profiles/panama', '/statistics/twitter/profiles/papua-new-guinea', '/statistics/twitter/profiles/paraguay', '/statistics/twitter/profiles/peru', '/statistics/twitter/profiles/philippines', '/statistics/twitter/profiles/poland', '/statistics/twitter/profiles/portugal', '/statistics/twitter/profiles/puerto-rico', '/statistics/twitter/profiles/qatar', '/statistics/twitter/profiles/republic-of-the-congo', '/statistics/twitter/profiles/reunion', '/statistics/twitter/profiles/romania', '/statistics/twitter/profiles/russia', '/statistics/twitter/profiles/rwanda', '/statistics/twitter/profiles/saint-kitts-and-nevis', '/statistics/twitter/profiles/saint-lucia', '/statistics/twitter/profiles/saint-martin', '/statistics/twitter/profiles/saint-vincent-and-the-grenadines', '/statistics/twitter/profiles/samoa', '/statistics/twitter/profiles/san-marino', '/statistics/twitter/profiles/sao-tome-and-principe', '/statistics/twitter/profiles/saudi-arabia', '/statistics/twitter/profiles/senegal', '/statistics/twitter/profiles/serbia', '/statistics/twitter/profiles/seychelles', '/statistics/twitter/profiles/sierra-leone', '/statistics/twitter/profiles/singapore', '/statistics/twitter/profiles/slovakia', '/statistics/twitter/profiles/slovenia', '/statistics/twitter/profiles/somalia', '/statistics/twitter/profiles/south-africa', '/statistics/twitter/profiles/south-korea', '/statistics/twitter/profiles/south-sudan', '/statistics/twitter/profiles/spain', '/statistics/twitter/profiles/sri-lanka', '/statistics/twitter/profiles/sudan', '/statistics/twitter/profiles/suriname', '/statistics/twitter/profiles/swaziland', '/statistics/twitter/profiles/sweden', '/statistics/twitter/profiles/switzerland', '/statistics/twitter/profiles/syria', '/statistics/twitter/profiles/taiwan', '/statistics/twitter/profiles/tajikistan', '/statistics/twitter/profiles/tanzania', '/statistics/twitter/profiles/thailand', '/statistics/twitter/profiles/togo', '/statistics/twitter/profiles/tokelau', '/statistics/twitter/profiles/tonga', '/statistics/twitter/profiles/trinidad-and-tobago', '/statistics/twitter/profiles/tunisia', '/statistics/twitter/profiles/turkey', '/statistics/twitter/profiles/turkmenistan', '/statistics/twitter/profiles/turks-and-caicos-islands', '/statistics/twitter/profiles/u-s-virgin-islands', '/statistics/twitter/profiles/uganda', '/statistics/twitter/profiles/ukraine', '/statistics/twitter/profiles/united-arab-emirates', '/statistics/twitter/profiles/united-kingdom', '/statistics/twitter/profiles/united-states', '/statistics/twitter/profiles/uruguay', '/statistics/twitter/profiles/uzbekistan', '/statistics/twitter/profiles/vanuatu', '/statistics/twitter/profiles/vatican', '/statistics/twitter/profiles/venezuela', '/statistics/twitter/profiles/vietnam', '/statistics/twitter/profiles/yemen', '/statistics/twitter/profiles/zambia', '/statistics/twitter/profiles/zimbabwe']
print(len(active_list),active_list)

231 ['/statistics/twitter/profiles/afghanistan', '/statistics/twitter/profiles/aland-islands', '/statistics/twitter/profiles/albania', '/statistics/twitter/profiles/algeria', '/statistics/twitter/profiles/american-samoa', '/statistics/twitter/profiles/andorra', '/statistics/twitter/profiles/angola', '/statistics/twitter/profiles/anguilla', '/statistics/twitter/profiles/antigua-and-barbuda', '/statistics/twitter/profiles/argentina', '/statistics/twitter/profiles/armenia', '/statistics/twitter/profiles/aruba', '/statistics/twitter/profiles/australia', '/statistics/twitter/profiles/austria', '/statistics/twitter/profiles/azerbaijan', '/statistics/twitter/profiles/bahamas', '/statistics/twitter/profiles/bahrain', '/statistics/twitter/profiles/bangladesh', '/statistics/twitter/profiles/barbados', '/statistics/twitter/profiles/belarus', '/statistics/twitter/profiles/belgium', '/statistics/twitter/profiles/belize', '/statistics/twitter/profiles/benin', '/statistics/twitter/profiles/bermuda', 

### Getting Profile Ext List (5 mins)

In [8]:
# Scours each countries' page to get the top profiles and put them in a list
profile_ext_list = []
start = time.time()
count = 0
for i in active_list:
  profile_ext_list.append(get_profile_ext(i))
  if count % 10 == 0:
    print(count,'out of',len(active_list),' countries completed.')
  count+=1
end = time.time()
print("Time Taken: {:.6f}s".format(end-start))
profile_ext_list = list(set(chain.from_iterable(profile_ext_list)))
print(len(profile_ext_list),profile_ext_list)

0 out of 231 done.
10 out of 231 done.
20 out of 231 done.
30 out of 231 done.
40 out of 231 done.
50 out of 231 done.
60 out of 231 done.
70 out of 231 done.
80 out of 231 done.
90 out of 231 done.
100 out of 231 done.
110 out of 231 done.
120 out of 231 done.
130 out of 231 done.
140 out of 231 done.
150 out of 231 done.
160 out of 231 done.
170 out of 231 done.
180 out of 231 done.
190 out of 231 done.
200 out of 231 done.
210 out of 231 done.
220 out of 231 done.
230 out of 231 done.
Time Taken: 246.584835s
2671 ['...']


### Getting Static Profile Ext List [Start here if you recently ran (and saved output for) above code]

In [9]:
# Saved output from the previous time I ran above code to save time
static_profile_ext_list = ['...']

try:
  profile_ext_list = profile_ext_list
except:
  profile_ext_list = static_profile_ext_list

### Getting Chart Link List

In [10]:
# Obtains chart links from the profile links
chart_link_list = []
start = time.time()
for i in profile_ext_list:
  chart_link_list.append(get_chart_link(i))
end = time.time()
print("Time Taken: {:.6f}s".format(end-start))
chart_link_list = list(set(chart_link_list))
print(len(chart_link_list),chart_link_list)

Time Taken: 0.004313s
2671 ['https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids=439164730&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months', 'https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids=2511441026&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months', 'https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids=538319374&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months', 'https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids=14568534&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months', 'https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids=54680554&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months', 'https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids=61888243&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months', 'https://api-sbks.socialbakers.com/charts/twitter/followers.html?ids=221066268&colors=45ada8%3Ba7c686%3B547980&interval=last_6_months', 'https://api-sbks.soci

### Get User-Follower List (20 mins)

In [11]:
# Scraps the followers from the list of profiles from social bakers website - Default condition set to obtain profiles with more than 10,000 followers and have consistent follower growth over (2w,1m,3m,6m)
driver = webdriver.Chrome('chromedriver',options=chrome_options)
output = dict()
count = 1
start = time.time()

for i in chart_link_list:
  profile_ext = i.split('?ids=')[1].split('&colors=')[0]
  try:
    _, follower_list, num_list = get_followers(i)
    if follower_list[-1] >= 10000 and follower_list[6]>follower_list[0] and follower_list[10]>follower_list[6] and follower_list[11]>follower_list[10]: # amend conditions here
      pct_chg_list, followers_bf_list, followers_af = pct_chg(follower_list, num_list)
      pct_chg_list.append(followers_af)
      output[profile_ext] = pct_chg_list
      #print(profile_ext, str(round(output[profile_ext][0]*100,2)) + '% growth over 6m, currently has ' + str(followers_af) + ' followers')
  except:
    pass
    #print("NO DATA")
  if count % 100 == 0:
    print(str(count)+" out of " + str(len(chart_link_list)) + " completed.", len(output),"inserted")
  count += 1      
end = time.time()
print("Time Taken: {:.6f}s".format(end-start))
print(len(output),output)

100 out of 2671 completed. 42 inserted
200 out of 2671 completed. 87 inserted
300 out of 2671 completed. 126 inserted
400 out of 2671 completed. 171 inserted
500 out of 2671 completed. 202 inserted
600 out of 2671 completed. 234 inserted
700 out of 2671 completed. 282 inserted
800 out of 2671 completed. 325 inserted
900 out of 2671 completed. 362 inserted
1000 out of 2671 completed. 398 inserted
1100 out of 2671 completed. 434 inserted
1200 out of 2671 completed. 477 inserted
1300 out of 2671 completed. 522 inserted
1400 out of 2671 completed. 566 inserted
1500 out of 2671 completed. 607 inserted
1600 out of 2671 completed. 642 inserted
1700 out of 2671 completed. 684 inserted
1800 out of 2671 completed. 718 inserted
1900 out of 2671 completed. 757 inserted
2000 out of 2671 completed. 796 inserted
2100 out of 2671 completed. 827 inserted
2200 out of 2671 completed. 865 inserted
2300 out of 2671 completed. 904 inserted
2400 out of 2671 completed. 950 inserted
2500 out of 2671 completed.

### Getting Static User-Follower List [Start here if you want to use the same followers figures

In [15]:
#done in 15 mins
static_output = {'439164730': [0.18251, 0.0732, 0.02586, 0.01102, 4458028], '2511441026': [0.0168, 0.0134, 0.00901, 0.00791, 725995], '19084628': [0.03106, 0.01278, 0.00307, 0.00175, 873112], '121887978': [0.05701, 0.05089, 0.00196, -0.00057, 1051688], '700961526': [0.06204, 0.02972, 0.01132, 0.00403, 1756042], '73877533': [0.22737, 0.08923, 0.0301, 0.01459, 240059], '622868850': [0.09616, 0.0394, 0.01128, 0.00531, 650604], '35697740': [0.12846, 0.04788, 0.01644, 0.00899, 950742], '950341134': [0.051, 0.02324, 0.00278, 0.00246, 7941437], '278934052': [0.01684, 0.00867, 0.00458, 0.00142, 309647], '2828470096': [0.08255, 0.04075, 0.01082, 0.00495, 855736], '172425386': [244936.0, 0.01948, 0.00691, 0.00335, 244937], '945310303': [0.02262, 0.01332, 0.00423, 0.00118, 841689], '2216589010': [269956.0, 0.14956, 0.01505, 0.00372, 269957], '2882944579': [0.07944, 0.0484, 0.01912, 0.00804, 572154], '429330888': [0.05763, 0.02829, 0.0128, 0.00496, 193256], '382673029': [0.05075, 0.01785, 0.00174, 0.0016, 14368], '2242189550': [0.26356, 0.25205, 0.1249, 0.1033, 103761], '49632982': [0.10607, 0.07716, 0.02677, 0.01281, 1120895], '462787588': [0.14232, 0.02999, 0.00293, 0.00218, 18822], '1317925608': [0.55667, 0.30057, 0.08587, 0.04021, 1013992], '3448833448': [0.05704, 0.03089, 0.00432, 0.00337, 76187], '78681704': [0.07109, 0.02478, 0.00645, 0.00263, 704752], '931157413331337216': [0.04348, 0.0256, 0.02501, 0.01735, 80237], '86899675': [0.04866, 0.02776, 0.0132, 0.00234, 3751643], '312667707': [0.07341, 0.03611, 0.00982, 0.0056, 10587], '284570882': [0.10098, 0.05762, 0.01682, 0.01264, 1499719], '47933779': [0.18047, 0.11643, 0.03291, 0.01412, 37913], '3898949008': [0.13221, 0.06051, 0.01328, 0.00687, 13197], '1059325677202157568': [0.34901, 0.12505, 0.04001, 0.0176, 1632911], '714377713': [0.26448, 0.11849, 0.03203, 0.01632, 558304], '927741488': [0.3131, 0.15824, 0.0431, 0.03364, 15488], '281947148': [0.08406, 0.0515, 0.00647, 0.00207, 632118], '1362191299': [0.07379, 0.04559, 0.0128, 0.00167, 1283205], '61454352': [0.0842, 0.02111, 0.00843, 0.00131, 41269], '2283571250': [0.05171, 0.0282, 0.00545, 0.00453, 14401], '16332498': [0.11837, 0.0382, 0.00986, 0.0048, 419096], '1174256078': [0.07339, 0.03587, 0.01015, 0.00466, 203771], '336025782': [0.07038, 0.04118, 0.01476, 0.00639, 1994613], '234240620': [0.10418, 0.05722, 0.01153, 0.00481, 10440], '105570727': [0.20008, 0.09257, 0.03088, 0.00935, 955809], '347349823': [0.01677, 0.01103, 0.00499, 0.00458, 421564], '111859266': [0.0146, 0.00712, 0.00137, 0.00052, 618927], '3119992756': [0.11946, 0.0469, 0.01781, 0.00509, 999245], '1317424410': [0.12207, 0.05471, 0.01451, 0.00727, 72773], '168259012': [0.10226, 0.04634, 0.01497, 0.00757, 840873], '178931425': [143343.0, 0.17472, 0.12557, 0.04002, 143344], '133379712': [0.07379, 0.02261, 0.00206, 0.00138, 252686], '78502161': [0.09615, 0.06934, 0.02174, 0.017, 2545706], '2805381283': [0.13818, 0.04338, 0.01058, 0.00489, 18904], '2936714848': [0.05738, 0.02092, 0.00567, 0.00193, 4101284], '292908967': [0.07524, 0.03133, 0.00315, 0.0024, 20049], '87027640': [0.7669, 0.42797, 0.11571, 0.05048, 1152923], '19280428': [0.01205, 0.00466, 0.00086, 0.00055, 611276], '179937965': [0.24778, 0.16276, 0.06307, 0.01077, 79149], '343627165': [0.10415, 0.04957, 0.0146, 0.00726, 26800873], '36406535': [0.03269, 0.02188, 0.00346, 0.00151, 27228], '3144380667': [0.06062, 0.02149, 0.00489, 0.00244, 12737], '226944459': [0.19704, 0.07465, 0.02149, 0.00947, 3492165], '22910295': [0.08875, 0.04603, 0.01583, 0.00969, 17116950], '256747696': [0.0566, 0.02192, 0.00471, 0.00214, 1574209], '64681939': [0.11058, 0.037, 0.00994, 0.00577, 42180], '284694804': [0.12767, 0.09187, 0.06161, 0.02447, 2421689], '71783170': [0.02262, 0.00527, 4e-05, 0.0, 1953940], '2341888057': [0.19035, 0.07031, 0.01831, 0.00859, 4842517], '381141996': [0.07176, 0.02654, 0.0086, 0.00618, 285457], '560641368': [0.0265, 0.01271, 0.00404, 0.00199, 1484716], '3092444379': [0.19743, 0.08983, 0.02645, 0.01149, 30817], '2274984734': [68489.0, 0.10405, 0.005, 0.00352, 68490], '742699047941771264': [0.16553, 0.0588, 0.01955, 0.0108, 134810], '1054615387': [0.13483, 0.06075, 0.02585, 0.01293, 153121], '115133899': [0.03286, 0.02028, 0.01056, 0.00573, 156985], '111240404': [0.02793, 0.0151, 0.00561, -0.00409, 948911], '2954204154': [0.09466, 0.02483, 0.00603, 0.0037, 1089147], '820190174': [0.0945, 0.05858, 0.01671, 0.00433, 63828], '15958614': [0.20454, 0.05817, 0.02282, 0.01012, 171550], '4873308778': [0.26491, 0.12834, 0.03191, 0.01487, 1590100], '601911813': [0.07568, 0.02791, 0.00881, 0.00598, 126190], '3437782151': [0.30602, 0.161, 0.03697, 0.01878, 45243], '1395958212': [0.11491, 0.0743, 0.0464, 0.01449, 41654], '1365017234': [0.17208, 0.1136, 0.06719, 0.03812, 467695], '30352096': [0.3459, 0.13034, 0.01599, 0.01075, 520444], '726757620057563136': [2.14732, 0.71272, 0.06714, 0.01578, 66421], '240180498': [0.0806, 0.0642, 0.02758, 0.01419, 1238993], '869586962': [0.06822, 0.0272, 0.00352, 0.00088, 10272], '395305090': [0.07012, 0.02543, 0.00992, 0.00375, 2470093], '448165069': [0.02791, 0.01131, 0.00239, 0.00132, 605864], '201208124': [0.14724, 0.09157, 0.03682, 0.0173, 83481], '34743251': [0.24716, 0.10832, 0.03218, 0.01132, 17145761], '40850617': [0.14748, 0.05932, 0.01677, 0.00894, 233383], '26014321': [0.04631, 0.02381, 0.01235, 0.00774, 148235], '1864123538': [0.02486, 0.01078, 0.00151, 0.00103, 326648], '2754746065': [0.48333, 0.15959, 0.03936, 0.0188, 4487475], '1026420680227741697': [0.22583, 0.08718, 0.02137, 0.00965, 21126], '16477279': [0.0834, 0.02611, 0.00618, 0.00378, 349275], '497665754': [0.05973, 0.0304, 0.01925, 0.00804, 2240698], '237135272': [0.06654, 0.02913, 0.00965, 0.00353, 561060], '1448229596': [0.04886, 0.02138, 0.00592, 0.00379, 22259], '989532019': [0.12337, 0.05908, 0.01358, 0.00667, 133481], '19236297': [0.05531, 0.02306, 0.0084, 0.00525, 1090312], '757487603562217472': [0.08512, 0.02704, 0.01311, 0.00443, 127965], '53189543': [0.1102, 0.10285, 0.05612, 0.04344, 1182706], '385282961': [0.14105, 0.07724, 0.02598, 0.01036, 275698], '1525201484': [0.05448, 0.02313, 0.00581, 0.00361, 11942], '787182156': [0.07564, 0.03043, 0.00695, 0.00278, 29253], '1432893792': [0.04271, 0.03299, 0.00863, 0.00224, 806068], '715201886442373120': [1.43424, 0.82037, 0.32558, 0.16081, 18342], '14527699': [0.98908, 0.49085, 0.15535, 0.05799, 76321], '53059376': [0.07856, 0.04445, 0.02644, 0.00853, 5468049], '575416734': [0.01059, 0.00421, 0.00063, 0.00056, 533419], '226813577': [0.11141, 0.05397, 0.02307, 0.02138, 362306], '124165495': [0.03623, 0.01369, 0.00308, 0.0017, 749796], '2577349345': [0.02203, 0.01434, 0.0107, 0.00958, 314012], '3005931631': [0.1995, 0.12524, 0.05162, 0.02489, 183107], '37601149': [0.05509, 0.02056, 0.00469, 0.00232, 2353581], '262230432': [0.30372, 0.22405, 0.07826, 0.03319, 1096058], '2941729382': [0.07009, 0.03493, 0.00905, 0.00305, 474443], '248825507': [0.57977, 0.18836, 0.0595, 0.02655, 449483], '20837438': [0.00819, 0.0026, 0.00089, 0.00019, 7150645], '170396389': [0.20057, 0.0514, 0.02099, 0.00745, 491857], '157725539': [0.24425, 0.09532, 0.02894, 0.01358, 77676], '84311137': [0.0578, 0.03389, 0.00561, 0.00145, 1333603], '3351760203': [0.0659, 0.02214, 0.00481, 0.00217, 414950], '139092348': [0.02057, 0.01118, 0.00661, 0.00587, 14209747], '74452681': [0.08427, 0.04787, 0.02097, 0.00888, 290841], '923434183': [0.33412, 0.13337, 0.03846, 0.01961, 186041], '873115441303924736': [0.15608, 0.06711, 0.0262, 0.01327, 11401093], '376914446': [0.01288, 0.01004, 0.00565, 1e-05, 227776], '245540045': [0.07974, 0.0319, 0.00946, 0.00498, 69584], '306974174': [0.06011, 0.01826, 0.00356, 0.00279, 45326], '3238343020': [0.11856, 0.03901, 0.00695, 0.00331, 100465], '13298072': [0.52076, 0.20988, 0.07127, 0.02145, 10218444], '1903196666': [0.48357, 0.25508, 0.04816, 0.02212, 85593], '423184716': [0.08962, 0.03891, 0.01103, 0.00445, 659830], '20736511': [0.16996, 0.10901, 0.05978, 0.04896, 2637849], '28684404': [0.06308, 0.02597, 0.00602, 0.00344, 754300], '2830604556': [0.17919, 0.03206, 0.00672, 0.0036, 2166427], '22361946': [0.21046, 0.11547, 0.02496, 0.01038, 72877], '3398216909': [0.05243, 0.02785, 0.01367, 0.00502, 965492], '78879048': [0.28566, 0.09205, 0.02738, 0.01281, 1746227], '401579858': [0.0953, 0.05138, 0.02003, 0.00891, 1217653], '1939017829': [0.12695, 0.05442, 0.02697, 0.01031, 455656], '630605295': [0.14895, 0.10376, 0.03637, 0.01808, 769685], '2333223628': [0.18954, 0.08072, 0.02891, 0.01189, 2578539], '1021725831515852800': [0.04518, 0.01331, 0.00458, 0.00195, 500272], '140595370': [0.03319, 0.01004, 0.00137, 0.00105, 28579], '263074291': [0.21629, 0.09544, 0.03532, 0.01387, 231384], '31593405': [0.42535, 0.33668, 0.0907, 0.04023, 27200], '291000010': [0.11161, 0.05091, 0.02284, 0.01975, 56741], '279966754': [0.0139, 0.00691, 0.00407, 0.00392, 1554877], '748291920': [0.10793, 0.0482, 0.0177, 0.00717, 465903], '4832636513': [0.17235, 0.09225, 0.04394, 0.02022, 219861], '856025974735085569': [0.3448, 0.16258, 0.05082, 0.02268, 190947], '300348359': [0.02694, 0.01074, 0.00074, 0.00046, 21648], '9075022': [0.06683, 0.03503, 0.01585, 0.01016, 2081379], '321180322': [0.00914, 0.00372, 0.00082, 0.00021, 3662923], '382671909': [0.1103, 0.04665, 0.00956, 0.00551, 201138], '56052218': [0.06943, 0.02775, 0.00889, 0.00455, 290966], '85346528': [0.07499, 0.02588, 0.00595, 0.00339, 365714], '7841202': [0.03677, 0.01796, 0.00387, 0.00185, 121937], '41585668': [0.0752, 0.03875, 0.0075, 0.00636, 13297], '138457648': [0.01295, 0.00494, 0.00107, 0.00074, 242687], '543770623': [0.03521, 0.01369, 0.00402, 0.0022, 533295], '1638021792': [0.15573, 0.06129, 0.02149, 0.01113, 2506148], '1649958450': [0.11787, 0.0493, 0.00974, 0.00407, 4960702], '1083808132067983365': [0.27091, 0.12625, 0.0326, 0.01198, 46307], '461112276': [0.01415, 0.00538, 0.0021, 0.00111, 3402391], '49953204': [0.08042, 0.06111, 0.03305, 0.03117, 15316], '200267797': [0.01848, 0.00772, 0.00077, 0.00073, 3753600], '622777464': [0.02864, 0.01423, 0.00349, 0.00126, 3464133], '29677101': [0.0585, 0.04643, 0.02243, 0.00304, 13860], '271103307': [0.10662, 0.0502, 0.01377, 0.00671, 57575], '2305870420': [0.1585, 0.08856, 0.01561, 0.00337, 403183], '452830010': [0.04298, 0.02519, 0.00666, 0.00318, 7343303], '3072209228': [0.26784, 0.19896, 0.06232, 0.03581, 1666842], '114894966': [0.0629, 0.03631, 0.01782, 0.00466, 571705], '2449336058': [0.10419, 0.05952, 0.01243, 0.00776, 37294], '2409683900': [0.04836, 0.02421, 0.01084, 0.00809, 442785], '136002930': [0.00619, 0.00239, 0.00076, 0.00012, 14697793], '2416154628': [0.1427, 0.08747, 0.0377, 0.01012, 708167], '2550023839': [0.066, 0.03781, 0.01413, 0.00865, 995514], '972804638': [0.04127, 0.01777, 0.00349, 0.00148, 253372], '298457633': [0.20758, 0.08494, 0.03073, 0.01241, 2275930], '1086378912': [0.18072, 0.05973, 0.01748, 0.01132, 233000], '177978853': [0.11706, 0.04613, 0.01281, 0.00652, 159727], '1372552416': [0.11349, 0.08273, 0.04264, 0.03107, 204711], '158487331': [0.05429, 0.01926, 0.00251, 0.00125, 53713443], '279367490': [0.05803, 0.02331, 0.00912, 0.00549, 1473093], '132880407': [0.11587, 0.08231, 0.02524, 0.00839, 1378655], '1708637154': [0.10509, 0.0442, 0.01598, 0.00921, 186626], '334032462': [0.11073, 0.04422, 0.01273, 0.0067, 428866], '46570604': [0.158, 0.1211, 0.07415, 0.00839, 83686], '130039824': [0.06266, 0.02041, 0.00298, 0.00212, 251035], '538285556': [0.02241, 0.01226, 0.0065, 0.00165, 2235172], '49849732': [0.06998, 0.05486, 0.0144, 0.00504, 4303442], '949415239': [0.10331, 0.0356, 0.00665, 0.00297, 46636], '3129056526': [0.15778, 0.06259, 0.02311, 0.01242, 8016305], '26180582': [0.26966, 0.26344, 0.24535, 0.23136, 631722], '84144260': [0.06657, 0.03239, 0.0099, 0.00603, 14692], '220906523': [0.04553, 0.02923, 0.01397, 0.00547, 127132], '51452048': [0.02902, 0.0145, 0.00745, 0.00527, 157772], '4827016745': [0.10129, 0.02276, 0.00555, 0.00252, 1377448], '58048133': [0.1065, 0.05711, 0.0157, 0.00838, 1708190], '154294030': [0.02249, 0.0179, 0.00358, 0.00087, 424282], '58487473': [1.18281, 0.86388, 0.25842, 0.078, 248386], '18424289': [0.06019, 0.03647, 0.01698, 0.00211, 1932779], '135070832': [0.11848, 0.03988, 0.0084, 0.00505, 25687], '423384542': [0.1658, 0.10431, 0.03441, 0.01495, 6432652], '19483191': [0.07649, 0.03518, 0.0095, 0.00541, 214319], '2515899612': [0.09462, 0.03718, 0.00984, 0.0055, 239636], '70155097': [0.00485, 0.00097, -0.00125, -0.00168, 610670], '3013693201': [0.09074, 0.04117, 0.00988, 0.00521, 227665], '3306155129': [0.05614, 0.02286, 0.00674, 0.00319, 378090], '70703740': [0.04037, 0.03215, 0.00777, 0.00245, 73580], '86444881': [0.02725, 0.01788, 0.01006, 0.00562, 371090], '902915467790225410': [0.48455, 0.24557, 0.06516, 0.02307, 82884], '563279467': [0.12892, 0.03957, 0.01344, 0.00693, 21935], '3397258065': [0.2351, 0.17507, 0.09677, 0.06652, 827017], '223421619': [0.23708, 0.06589, 0.01858, 0.00837, 39521], '2805358944': [0.01919, 0.00937, 0.00275, 0.00138, 3122233], '1487093702': [0.06139, 0.02195, 0.00708, 0.0035, 1479183], '179832889': [0.04203, 0.01878, 0.00301, 0.00193, 21320], '3177475255': [0.02941, 0.01437, 0.00595, 0.00303, 778764], '906771091': [0.01542, 0.0085, 0.00374, 0.00127, 716692], '17128975': [0.2958, 0.10512, 0.03187, 0.01467, 1835163], '1144935463': [0.0692, 0.02895, 0.00647, 0.00322, 468037], '183253150': [0.0556, 0.02434, 0.0072, 0.00359, 3246343], '195014153': [0.01884, 0.00858, 0.00194, -0.00321, 721259], '42072558': [0.23745, 0.13338, 0.10003, 0.00511, 13383], '1299489810': [0.13049, 0.05627, 0.01736, 0.00854, 66502], '95217117': [0.03494, 0.01399, 0.00253, 0.00137, 2299468], '1105470817': [0.02448, 0.0112, 0.00539, 0.00281, 1356301], '320745703': [0.07688, 0.03121, 0.0108, 0.00657, 36771], '243851501': [0.18023, 0.0747, 0.02237, 0.00988, 1859826], '2418180198': [0.16499, 0.09846, 0.06297, 0.05156, 432125], '64245261': [0.18697, 0.07382, 0.02135, 0.01045, 90156], '103021125': [0.05675, 0.02637, 0.00781, 0.00488, 18957], '451772913': [0.10318, 0.04351, 0.01144, 0.0055, 667771], '110758650': [0.12863, 0.04195, 0.00685, 0.00521, 22207], '21387239': [28769.0, 0.026, 0.00587, 0.00352, 28770], '74618166': [0.4066, 0.05406, 0.00731, 0.00471, 26032], '3191855185': [0.11583, 0.04058, 0.01792, 0.00964, 298664], '32836955': [0.03736, 0.01852, 0.00324, 0.00245, 55756], '462004145': [0.00775, 0.0021, 0.0001, -1e-05, 8478102], '2495199115': [0.01035, 0.00704, 0.00494, 0.00491, 3919847], '27131277': [0.0482, 0.01996, 0.00175, 8e-05, 2091706], '471861846': [0.02618, 0.02167, 0.01597, 0.00123, 401055], '38164846': [0.07622, 0.03456, 0.00702, 0.00341, 9135862], '330446733': [0.048, 0.02902, 0.00554, 0.00289, 9118458], '117356282': [0.11622, 0.05654, 0.02365, 0.00663, 29303], '60576219': [0.05616, 0.02596, 0.01085, 0.00425, 10324586], '73981088': [0.0101, 0.00972, 0.00546, 0.00131, 899201], '1467280604': [0.28101, 0.19477, 0.01956, 0.00526, 26745], '14277989': [0.19066, 0.10679, 0.03285, 0.0142, 61337], '442918531': [0.26209, 0.10679, 0.02186, 0.00857, 49653], '1156898177246646272': [0.40744, 0.11718, 0.03215, 0.01388, 169556], '1580603696': [0.11839, 0.06531, 0.02769, 0.00873, 1133572], '138245609': [0.11086, 0.05933, 0.01389, 0.00613, 22336], '862027812': [0.08264, 0.06586, 0.02022, 0.01053, 93691], '344718893': [0.15526, 0.06606, 0.02531, 0.01368, 808901], '4821435801': [0.17405, 0.08035, 0.01575, 0.00783, 172647], '2946129982': [0.01491, 0.00835, 0.0025, 0.00075, 16064], '18187991': [0.08182, 0.02664, 0.00508, 0.00357, 16607], '2229718597': [0.14595, 0.06534, 0.02331, 0.00652, 1043253], '1176546593861820416': [0.78201, 0.38124, 0.16325, 0.07036, 34473], '823219165': [0.04634, 0.01676, 0.00504, 0.0031, 25539], '1322632638': [0.04072, 0.03266, 0.01809, 0.00706, 1060831], '82955132': [0.19687, 0.09243, 0.0273, 0.01308, 1831342], '296124687': [0.03599, 0.01553, 0.01269, 0.00516, 1355640], '38990369': [0.07198, 0.03517, 0.0077, 0.00358, 1749036], '760402465992929281': [0.1651, 0.07005, 0.02125, 0.01301, 54057], '2287904017': [0.98373, 0.56557, 0.14896, 0.04576, 36816], '2662529568': [0.08074, 0.03166, 0.01056, 0.00557, 30143], '126571977': [0.00957, 0.00361, 0.0007, 0.00039, 14939732], '3247896487': [0.06266, 0.03941, 0.01374, 0.00571, 1017319], '299582587': [0.22284, 0.06989, 0.00828, 0.00373, 11573], '736418855006920708': [0.0617, 0.02576, 0.01145, 0.0012, 47424], '1191105769': [0.07638, 0.01703, 0.00132, 0.00057, 314852], '2530363285': [0.36408, 0.16439, 0.04665, 0.02261, 839225], '628808642': [0.07375, 0.03792, 0.00775, 0.00343, 3340919], '360699754': [0.03537, 0.02477, 0.00497, 0.00338, 273805], '554534767': [0.07787, 0.03526, 0.00916, 0.00364, 1543532], '723866725': [0.11257, 0.06715, 0.03174, 0.02585, 587002], '2476130260': [0.21984, 0.051, 0.01009, 0.00331, 52753], '627673190': [0.09763, 0.04743, 0.01342, 0.00693, 34914737], '274322871': [0.09287, 0.04637, 0.03148, 0.01477, 1214169], '123042848': [0.0008, 0.00043, -0.00051, -0.00191, 241328], '450605189': [0.22891, 0.04631, 0.00558, 0.00239, 32646], '16882261': [0.188, 0.09108, 0.02066, 0.0109, 1822049], '46623193': [0.03221, 0.00987, 0.00357, -0.00049, 477299], '845250133': [0.09121, 0.02291, 0.007, 0.00398, 671932], '176379354': [0.09459, 0.04948, 0.01126, 0.00473, 11051], '96735431': [0.05067, 0.0246, 0.00632, 0.00317, 386623], '1544328109': [0.06273, 0.02959, 0.00413, 0.00132, 17500], '70394965': [0.10759, 0.05615, 0.01546, 0.00705, 3271500], '258856900': [0.07735, 0.01997, 0.00218, -0.00186, 84583], '247653244': [0.08087, 0.04215, 0.01288, 0.00494, 7615970], '2626658949': [0.05748, 0.0271, 0.00797, 0.00438, 29105], '35180265': [0.01606, 0.00984, 0.00451, 0.00287, 78592], '3045608319': [0.36645, 0.16625, 0.04112, 0.02263, 869532], '833273612': [0.0782, 0.06646, 0.03635, 0.03203, 47816], '537204717': [1.02945, 0.16793, 0.02543, 0.00942, 459608], '2645440044': [0.16343, 0.0956, 0.04719, 0.03615, 352223], '44196397': [0.38528, 0.15693, 0.05001, 0.01502, 56875053], '434071161': [0.04068, 0.02054, 0.00473, 0.00206, 5753110], '1568130360': [0.04067, 0.02147, 0.00961, 0.0046, 946215], '299918176': [0.03513, 0.01792, 0.00956, 0.00703, 407806], '263096349': [0.01972, 0.00919, 0.00253, 0.00116, 406150], '1024926798': [0.02117, 0.01756, 0.01416, 0.0018, 3268076], '70988996': [0.07449, 0.04075, 0.01226, 0.00639, 10242], '1484391258': [0.03464, 0.01091, 0.00485, 0.00253, 39551], '108543358': [0.07459, 0.03821, 0.01041, 0.00498, 5768164], '579747564': [0.0343, 0.01409, 0.00192, -0.00022, 109144], '790921536': [0.01968, 0.01527, 0.00749, 0.00616, 223595], '165796189': [0.18316, 0.09742, 0.0282, 0.01253, 3036286], '76294023': [0.80092, 0.09805, 0.01904, 0.00746, 649315], '219294551': [0.08828, 0.05934, 0.02632, 0.01762, 2342066], '13120642': [0.08995, 0.04174, 0.01694, 0.00823, 77746], '23758430': [0.04635, 0.02211, 0.00932, 0.0046, 1479525], '188656529': [0.06547, 0.04185, 0.02488, 0.0115, 132628], '570223846': [0.06125, 0.01758, 0.00463, 0.00243, 643332], '217461138': [0.05509, 0.02983, 0.01962, 0.01318, 242896], '4767688869': [0.41776, 0.19752, 0.06122, 0.05017, 18964], '141235731': [0.03993, 0.01964, 0.00975, 0.00695, 130752], '419107960': [0.2082, 0.15232, 0.0239, 0.00824, 4925434], '1058748734820044800': [1838495.0, 1.24557, 0.15571, 0.05316, 1838496], '50981729': [0.22067, 0.17018, 0.05357, 0.01731, 1218317], '5536782': [0.04512, 0.02429, 0.01259, 0.0036, 16524947], '478747112': [0.06438, 0.03991, 0.01507, 0.00294, 1173961], '2592325530': [0.23624, 0.13119, 0.02302, 0.00681, 402857], '814021329632559104': [0.90466, 0.44552, 0.15976, 0.0855, 1395164], '157253297': [0.05231, 0.0273, 0.01611, 0.01258, 21001], '230186518': [0.11677, 0.03918, 0.01281, 0.00745, 21643946], '308959719': [0.03937, 0.02055, 0.0094, 0.00483, 615788], '255707941': [0.9937, 0.91543, 0.4596, 0.45404, 525490], '587463737': [0.20173, 0.11169, 0.04582, 0.01373, 1526227], '14573900': [0.12629, 0.06888, 0.02264, 0.01106, 9968156], '4457831668': [0.06592, 0.02273, 0.00704, 0.00371, 65954], '318455359': [0.0254, 0.01083, 0.003, 0.00172, 218161], '794866087': [0.07021, 0.03739, 0.02065, 0.00378, 559603], '111392659': [0.02937, 0.00849, 0.00125, 0.00092, 1623791], '112440529': [0.10422, 0.04274, 0.0126, 0.00583, 134830], '233983755': [0.05361, 0.02154, 0.00506, 0.00245, 655699], '19923144': [0.05228, 0.02459, 0.00458, 0.00338, 33475357], '65707471': [0.32332, 0.19201, 0.04703, 0.01943, 48374], '14283525': [0.08104, 0.01976, 0.00136, 0.00104, 312291], '35031708': [0.11151, 0.05473, 0.01177, 0.0012, 2840563], '252119096': [0.04629, 0.01499, 0.0037, 0.00234, 410174], '548692500': [0.26559, 0.12399, 0.05466, 0.0294, 46885], '69137871': [0.40887, 0.19923, 0.04982, 0.02857, 68126], '307589981': [0.11988, 0.09675, 0.05747, 0.00914, 344220], '474886468': [0.0544, 0.02559, 0.00668, 0.00292, 14537984], '187474143': [0.03623, 0.01627, 0.00202, 0.00162, 222736], '229151808': [0.06266, 0.0338, 0.01207, 0.00656, 501270], '240131294': [0.09566, 0.05018, 0.01032, 0.00398, 27226], '23765365': [3.34931, 0.07969, 0.01523, 0.00658, 519086], '53459457': [0.44089, 0.41333, 0.38911, 0.38869, 91855], '2228188470': [0.1693, 0.05099, 0.00909, 0.00507, 19643], '2390095775': [0.05995, 0.02296, 0.00538, 0.00383, 120944], '505076949': [0.06477, 0.02336, 0.00462, 0.00201, 276011], '39941478': [0.09534, 0.0399, 0.00532, 0.0051, 503581], '203879413': [0.01855, 0.00521, 0.00055, 9e-05, 1219968], '782516870394109952': [0.7803, 0.31177, 0.10008, 0.05385, 1638542], '1111567384148692993': [0.23876, 0.07974, 0.01221, 0.00394, 104626], '49057844': [0.03531, 0.01234, -3e-05, -0.00052, 100683], '300789811': [0.04727, 0.02337, 0.00615, 0.00275, 3079871], '103026062': [0.1408, 0.05638, 0.01778, 0.00908, 103986], '2943463768': [0.05475, 0.03475, 0.00762, 0.00173, 119483], '729582793576042496': [0.23929, 0.10047, 0.03101, 0.0084, 875339], '600091311': [0.11503, 0.04636, 0.01359, 0.00821, 28868], '47813521': [0.02436, 0.00939, 0.00127, 0.00039, 1669041], '43064490': [0.3644, 0.10171, 0.08306, 0.05497, 224952], '69183155': [0.02946, 0.0092, 0.00234, 0.00147, 16940147], '257057832': [0.06447, 0.02599, 0.00722, 0.00355, 858599], '47033057': [0.05017, 0.01653, 0.00402, 0.00252, 3600397], '283523828': [0.06318, 0.05104, 0.01408, 0.00096, 1034336], '124029017': [0.02371, 0.00884, 0.00123, 0.00073, 692385], '4197264629': [0.57747, 0.23976, 0.04513, 0.01824, 19985], '242245578': [0.00819, 0.0029, 0.00046, 0.00037, 19941496], '18901882': [0.02418, 0.01647, 0.01039, 0.00398, 2611260], '2396677714': [0.06468, 0.02839, 0.00876, 0.00293, 14508873], '2279776304': [0.10161, 0.03399, 0.00966, 0.00536, 1566148], '229454266': [0.09621, 0.05428, 0.01125, 0.00646, 996016], '525394081': [0.02352, 0.00669, 0.00149, 0.00125, 471743], '339055436': [0.07884, 0.05542, 0.02784, 0.02766, 260584], '140107171': [0.07979, 0.04283, 0.01792, 0.00558, 18919], '150429478': [0.03768, 0.02103, 0.00345, 0.00138, 1924138], '2904114874': [0.02842, 0.01142, 0.00283, 0.00195, 71324], '1343292967': [0.23481, 0.10684, 0.04618, 0.02084, 22087], '2656039566': [0.06235, 0.03136, 0.01195, 0.00663, 68499], '24236390': [0.04721, 0.02892, 0.01501, 0.00887, 71898], '2326519866': [0.0532, 0.02817, 0.0067, 0.00138, 13067], '1346891520': [0.14278, 0.06116, 0.02085, 0.00984, 330180], '398306220': [0.06303, 0.02036, 0.0047, 0.00242, 16764720], '265839544': [0.15903, 0.11086, 0.0547, 0.01091, 659947], '2935773834': [0.59954, 0.34829, 0.03385, 0.01552, 11056], '2344209337': [0.38333, 0.11105, 0.02724, 0.01306, 1454266], '286018054': [0.50168, 0.43482, 0.23836, 0.02799, 505820], '253213737': [0.06322, 0.02703, 0.00808, 0.00369, 446444], '2308483814': [0.03055, 0.01578, 0.0073, 0.00363, 874438], '171421559': [0.02443, 0.00867, 0.00146, 0.00097, 4119146], '70301713': [0.11355, 0.05176, 0.01062, 0.00508, 133660], '884378714': [0.08277, 0.0295, 0.00803, 0.00421, 9537193], '350163849': [0.14245, 0.06993, 0.01597, 0.00907, 863200], '947872754': [0.07672, 0.03306, 0.01096, 0.00572, 369919], '431996598': [0.04224, 0.01359, 0.00048, 0.00026, 7601091], '42933426': [0.05435, 0.02838, 0.01074, 0.00536, 4172539], '23578176': [0.07357, 0.02184, 0.01466, 0.00707, 522928], '126955629': [0.07028, 0.02682, 0.009, 0.00504, 2202466], '1244602586': [0.05831, 0.0164, 0.00434, 0.00379, 12786237], '622886707': [0.04718, 0.02596, 0.00624, 0.00294, 22882], '2660980824': [0.16423, 0.08097, 0.02431, 0.01171, 37246], '2377774422': [0.07997, 0.04174, 0.009, 0.00355, 14126], '215052012': [0.13344, 0.05803, 0.01402, 0.00645, 5381437], '957560196': [0.0118, 0.00842, 0.00122, 0.00029, 1827438], '366987179': [0.04376, 0.0144, 0.00367, 0.00176, 15763938], '1286537048': [0.09223, 0.03087, 0.00975, 0.00481, 24643], '1424697198': [0.07653, 0.02972, 0.00632, 0.00277, 26093], '391160098': [134868.0, 0.05997, 0.02146, 0.01067, 134869], '369583954': [0.03154, 0.01685, 0.00116, 0.00074, 8716296], '146556805': [0.04442, 0.01798, 0.00327, 0.00179, 2158840], '20356963': [2.02671, 0.91662, 0.14948, 0.04957, 210238], '14697575': [0.05993, 0.02392, 0.0061, 0.00347, 3984656], '1242198056': [0.05968, 0.04311, 0.01319, 0.00681, 23811], '2641849360': [0.01038, 0.0036, 0.00055, 0.0003, 1529863], '190706828': [0.08845, 0.0629, 0.02303, 0.01324, 449492], '110681593': [0.02331, 0.00704, 0.00222, 0.0007, 7817848], '231579149': [0.19617, 0.09212, 0.0331, 0.01535, 13419696], '3236507891': [0.10102, 0.04007, 0.00917, 0.00462, 116432], '4850837842': [0.24452, 0.11209, 0.04129, 0.01259, 657429], '44489439': [0.02618, 0.00808, 0.00144, 0.00098, 592460], '41396492': [0.11283, 0.08311, 0.02628, 0.00941, 2235151], '1422951168': [0.19409, 0.09251, 0.02909, 0.01343, 8480976], '65549291': [0.03158, 0.01749, 0.0067, 0.00488, 3026968], '4514477775': [0.2053, 0.04584, 0.01207, 0.00589, 1536034], '447667853': [0.05357, 0.04368, 0.0169, 0.00426, 737857], '145346276': [0.12031, 0.07937, 0.03415, 0.02867, 1804874], '466508944': [0.20536, 0.08021, 0.02246, 0.01153, 94404], '4631536043': [0.10068, 0.02992, 0.00618, 0.00357, 495954], '519209472': [0.33602, 0.17966, 0.03054, 0.01498, 58555], '132171061': [0.05261, 0.01965, 0.00299, 0.00186, 1754436], '819116354926243840': [0.14952, 0.10224, 0.05879, 0.03379, 22241], '2815728328': [0.0249, 0.01472, 0.00908, 0.00657, 802966], '93589182': [0.02685, 0.01251, 0.00668, 0.00352, 1138154], '182181105': [0.06183, 0.02783, 0.00821, 0.00403, 848525], '742143': [0.05913, 0.02207, 0.00282, 0.00173, 32079203], '1401587503': [0.01785, 0.00535, 0.0015, 0.00111, 8167578], '1106069527': [0.29869, 0.20323, 0.02796, 0.00922, 32279], '1034320597': [0.2779, 0.15343, 0.02914, 0.01271, 11795], '168557941': [0.06146, 0.02643, 0.0084, 0.0042, 363300], '3306946201': [0.97065, 0.90688, 0.29522, 0.29028, 19269], '297925007': [0.06972, 0.03461, 0.01513, 0.00651, 16234], '92487173': [0.02298, 0.013, 0.00622, 0.00259, 8101379], '104927840': [0.06417, 0.02623, 0.00504, 0.00295, 216580], '177838266': [0.04584, 0.01645, 0.00457, 0.00259, 976191], '1648029396': [0.08604, 0.04775, 0.01154, 0.00254, 383642], '159149849': [0.04048, 0.01583, 0.00444, 0.0022, 278256], '1120499768': [0.08829, 0.0317, 0.01488, 0.01147, 119432], '84729044': [0.11971, 0.0606, 0.02557, 0.0105, 1935946], '515615282': [0.08045, 0.04314, 0.00579, 0.00251, 11993], '306340109': [0.05044, 0.03328, 0.01674, 0.00606, 143642], '269940376': [0.01896, 0.00936, 0.00272, 0.00104, 4773919], '47448563': [0.0518, 0.02146, 0.00567, 0.0033, 1151544], '191040022': [0.00698, 0.00156, -0.00144, -0.0018, 625761], '8225692': [0.05315, 0.02263, 0.00585, 0.0028, 2334157], '121228332': [0.03963, 0.01697, 0.00326, 0.00183, 42188], '984059580232687616': [0.2656, 0.10848, 0.03297, 0.01777, 160775], '1074358534853271555': [158513.0, 0.06689, 0.01904, 0.00975, 158514], '4887252526': [0.41692, 0.1986, 0.06639, 0.02505, 158577], '3096689986': [0.10346, 0.04659, 0.01089, 0.00549, 35558], '949934119': [0.51783, 0.46346, 0.40693, 0.26562, 29575], '23186079': [0.03964, 0.01381, 0.00501, 0.00132, 10022128], '309133460': [0.08274, 0.0447, 0.01616, 0.00595, 2377048], '1902330746': [0.10779, 0.05225, 0.02458, 0.01095, 15508], '581123471': [0.20888, 0.08243, 0.02844, 0.01529, 202323], '972597548199370752': [0.4583, 0.10988, 0.0269, 0.01398, 302700], '3029339866': [0.31557, 0.2237, 0.04915, 0.01249, 54883], '4794391108': [0.37454, 0.16832, 0.03785, 0.01657, 579708], '1244029188': [0.39857, 0.1368, 0.0332, 0.01505, 451310], '732521058507620356': [0.1443, 0.04888, 0.01229, 0.00577, 173011], '162825036': [0.04715, 0.02138, 0.00665, 0.00286, 4812925], '388784593': [0.02483, 0.00836, 0.00151, -0.00017, 305395], '333902735': [0.09837, 0.04256, 0.00712, 0.00244, 106468], '61443691': [0.17358, 0.04026, 0.00425, 0.00221, 17261], '531289835': [0.13441, 0.06496, 0.02229, 0.0133, 35491], '18308305': [0.12745, 0.06809, 0.01906, 0.00776, 1344511], '1614950090': [0.45922, 0.23843, 0.09247, 0.04214, 135684], '812627249446780928': [0.14227, 0.06427, 0.01494, 0.00655, 572380], '125346858': [0.07203, 0.03347, 0.01121, 0.00613, 4765763], '134196350': [0.07263, 0.00678, 0.00248, 0.00065, 709876], '38671739': [0.02847, 0.01444, 0.00709, 0.00304, 2122141], '127765218': [0.07358, 0.03626, 0.00395, 0.0022, 2354842], '2155215398': [0.02424, 0.01837, 0.0109, 0.00727, 5099208], '1092361470': [0.09076, 0.05538, 0.0173, 0.00561, 14349], '841211876776259585': [0.06102, 0.04271, 0.01799, 0.00357, 1997496], '41103933': [0.03764, 0.0171, 0.00388, 0.00205, 455654], '148711787': [0.0212, 0.01051, 0.00386, 0.00156, 14766011], '484987132': [0.07092, 0.04317, 0.01593, 0.00723, 350159], '44561223': [0.122, 0.10481, 0.07201, 0.06481, 211068], '226093367': [0.04611, 0.02077, 0.00744, 0.00442, 687288], '1769361': [0.05327, 0.02596, 0.00687, 0.00449, 126134], '370809101': [0.04505, 0.0212, 0.00309, 0.0022, 557010], '1575800623': [0.11016, 0.05395, 0.02031, 0.00775, 10552254], '77696645': [0.13583, 0.0809, 0.02789, 0.0128, 969131], '2799515463': [0.10568, 0.05188, 0.01209, 0.00675, 146964], '69008563': [0.13793, 0.10719, 0.02932, 0.01481, 5838490], '369406105': [0.2135, 0.09324, 0.02835, 0.00885, 14363], '330262748': [0.35612, 0.2435, 0.14148, 0.09286, 3071757], '533926992': [0.01126, 0.00536, 0.00093, 0.0005, 611277], '361445173': [0.0071, 0.00211, 0.00034, 0.0003, 545355], '544862804': [0.04233, 0.02345, 0.00889, 0.00133, 417360], '56951263': [0.15462, 0.11039, 0.03908, 0.01989, 624177], '467075090': [0.08827, 0.04323, 0.02842, 0.01292, 1148235], '31348594': [0.03566, 0.01482, 0.00243, 0.00126, 41598440], '301120393': [0.09935, 0.04472, 0.01703, 0.00797, 10835000], '140068649': [0.02274, 0.00807, 0.00117, 0.00074, 868284], '444969570': [0.19531, 0.07453, 0.02775, 0.00516, 52376], '3391713417': [0.0885, 0.04953, 0.00884, 0.00403, 59778], '141817380': [0.12468, 0.04011, 0.01039, 0.00508, 13988592], '459033261': [0.17131, 0.08648, 0.02754, 0.01428, 1095068], '86037380': [0.074, 0.03183, 0.00759, 0.00423, 1092947], '460876965': [0.05885, 0.01635, 0.00032, -0.00065, 117888], '1001022572186820608': [1.18225, 0.41357, 0.08406, 0.02562, 172485], '246967511': [0.08192, 0.04489, 0.00911, 0.0043, 3690087], '455680347': [0.06375, 0.01553, 0.00094, 0.00065, 13799], '2385272859': [0.0452, 0.03905, 0.01418, 0.00764, 272333], '21367580': [0.05997, 0.02851, 0.01525, 0.0104, 496787], '50091650': [0.02225, 0.00708, 0.00105, 0.00101, 784485], '228171408': [0.04499, 0.01618, 0.0038, 0.00202, 422602], '219858307': [0.10621, 0.04273, 0.01983, 0.01076, 3883179], '455895084': [0.14166, 0.0866, 0.03953, -0.00295, 163468], '1188735828886900737': [0.13999, 0.0969, 0.02804, 0.00643, 1025430], '460116210': [0.1225, 0.06315, 0.02876, 0.01846, 1264753], '29785926': [0.54553, 0.24312, 0.16224, 0.12853, 141620], '1573440108': [0.37921, 0.04276, 0.0123, 0.00589, 1299070], '1051795722': [0.41835, 0.18091, 0.07278, 0.04852, 528581], '1029323618': [0.17147, 0.06965, 0.02056, 0.01052, 1529958], '2430656918': [0.32268, 0.22295, 0.08896, 0.05029, 515067], '19624061': [0.0208, 0.00869, 0.00193, 0.00129, 825011], '299693451': [0.03687, 0.01237, 0.00295, 0.00186, 898827], '109220563': [0.10316, 0.03847, 0.01382, 0.00775, 1219567], '207511616': [0.43407, 0.16854, 0.03944, 0.01979, 85750], '803199792369180673': [0.09926, 0.04551, 0.01495, 0.00717, 2295756], '1371618385': [0.0157, 0.00916, 0.00456, 0.00228, 843500], '34890810': [0.0421, 0.01293, 0.00257, 0.00129, 1218168], '205622130': [0.09904, 0.03818, 0.00974, 0.0029, 944250], '1188260811854303232': [0.54088, 0.23863, 0.09806, 0.03583, 56313], '205605464': [0.09677, 0.02999, 0.00241, 0.00145, 64806], '848822905': [0.05622, 0.04942, 0.00413, 0.00067, 786253], '15007299': [0.01954, 0.00712, 0.00088, 0.00065, 5161754], '395425451': [0.06962, 0.02969, 0.00951, 0.00443, 50838], '2254854108': [0.08527, 0.02988, 0.01388, 0.00795, 149688], '219867859': [0.02448, 0.00868, 0.00098, 0.00081, 5044864], '225305750': [0.0155, 0.00617, 0.00154, 0.0007, 414805], '3023347061': [0.30868, 0.15086, 0.05508, 0.03007, 10688], '41786801': [0.1784, 0.09587, 0.0427, 0.01968, 140270], '37950212': [0.0168, 0.00884, 0.00195, 0.00019, 3001945], '343952969': [0.16041, 0.05094, 0.0125, 0.00623, 169622], '113286385': [0.09548, 0.05192, 0.01595, 0.00636, 4046719], '2232741553': [0.2181, 0.10582, 0.02969, 0.01044, 380155], '67507531': [0.25541, 0.13976, 0.05647, 0.01491, 2156159], '2223114825': [0.08951, 0.02756, 0.01422, 0.01109, 258686], '822003118921699329': [0.31873, 0.13102, 0.01387, 0.00617, 98748], '83658040': [0.12105, 0.05728, 0.02645, 0.01276, 111101], '75229951': [0.1113, 0.04681, 0.01524, 0.00745, 782528], '192309454': [0.02624, 0.01105, 0.00298, 0.0013, 210021], '2493201162': [0.10396, 0.04751, 0.01388, 0.00566, 36094], '789546061': [0.10026, 0.05086, 0.01303, 0.01084, 13059], '3297036587': [0.2096, 0.11292, 0.01815, 0.00634, 78780], '248883264': [0.11364, 0.04928, 0.00506, 0.00348, 939092], '3621289273': [0.15979, 0.0754, 0.0169, 0.00679, 190510], '1130891570': [0.09644, 0.0467, 0.02048, 0.00582, 748806], '145741832': [0.03604, 0.02241, 0.01167, 0.01015, 368892], '335141638': [0.14687, 0.0721, 0.02635, 0.01296, 36193946], '531041640': [0.34765, 0.13868, 0.08029, 0.03344, 1414089], '56960258': [0.0927, 0.03882, 0.01444, 0.00833, 778227], '145935897': [0.10525, 0.03928, 0.01259, 0.00583, 675119], '43791845': [0.02639, 0.00984, 0.00086, 0.00047, 277661], '4202917391': [0.18682, 0.08214, 0.01554, 0.00806, 18760], '26835989': [0.42471, 0.0341, 0.01215, 0.00371, 52495], '2563603760': [0.31936, 0.11408, 0.02546, 0.01442, 147043], '2457956456': [0.1527, 0.07948, 0.0315, 0.01649, 12576], '116393932': [0.04374, 0.0226, 0.00745, 0.00278, 4845423], '213798858': [0.08067, 0.05666, 0.02743, 0.0119, 604662], '1306925508': [0.06672, 0.03396, 0.00758, 0.0034, 43996], '1055490565': [0.01256, 0.0059, 0.00086, 0.00018, 1428391], '228822613': [0.08428, 0.02428, 0.0026, 0.00123, 1191212], '447803659': [0.34634, 0.33045, 0.17181, 0.01633, 133457], '153825949': [0.2627, 0.11691, 0.02563, 0.0136, 606583], '1102833698435674112': [0.16715, 0.10969, 0.0677, 0.02863, 35751], '81986272': [0.03256, 0.01341, 0.00308, 0.00173, 817302], '149733596': [0.03065, 0.01457, 0.0021, 0.00041, 1793907], '945571482': [0.05862, 0.02875, 0.00901, 0.00713, 38288], '97513349': [0.03995, 0.01476, 0.00254, 0.00147, 2588644], '556260847': [0.16794, 0.12953, 0.03839, 0.01939, 1708202], '44116121': [0.05651, 0.03625, 0.01091, 0.00472, 1716850], '64085882': [173674.0, 0.12069, 0.05347, 0.02819, 173675], '587257369': [0.02756, 0.0144, 0.00459, 0.0018, 576929], '55660696': [0.03612, 0.01422, 0.00408, 0.00283, 1354820], '73991669': [2.06521, 0.59, 0.10725, 0.02314, 590912], '75055756': [0.16192, 0.09646, 0.02484, 0.01058, 36476], '862675563693125632': [0.17087, 0.09912, 0.02256, 0.00629, 224366], '47930576': [0.0756, 0.03826, 0.00984, 0.00818, 25751], '2805500982': [0.12934, 0.06295, 0.02912, 0.01395, 15551], '1171334091637235714': [0.52128, 0.18675, 0.04326, 0.01312, 20621], '1059542139737128960': [3108183.0, 0.10906, 0.03293, 0.01828, 3108184], '233953901': [0.10503, 0.04627, 0.01627, 0.00821, 580852], '105252564': [0.17789, 0.06959, 0.01014, 0.00617, 23010], '1137984020': [0.08047, 0.03786, 0.00988, 0.00528, 2284506], '1366981790': [0.06484, 0.04651, 0.0179, 0.0087, 46512], '202608931': [2.54008, 0.29839, 0.04366, 0.01517, 160624], '37881369': [0.10562, 0.05888, 0.02225, 0.00831, 340633], '169274239': [0.03014, 0.01646, 0.00486, 0.00195, 7337681], '18469669': [0.42677, 0.19023, 0.07405, 0.03116, 133071], '575661707': [0.03123, 0.02311, 0.0136, 0.00254, 1276231], '96154796': [0.03939, 0.01601, 0.00603, 0.00178, 515758], '795280573761810433': [0.6115, 0.36881, 0.17677, 0.12888, 592181], '141250214': [0.07113, 0.02664, 0.00198, 0.00018, 22316], '322627086': [0.05858, 0.04209, 0.01862, 0.01062, 1042285], '237422158': [0.05182, 0.02391, 0.00596, 0.0048, 11306], '14872237': [0.05082, 0.02839, 0.00707, 0.00384, 37455703], '376733906': [0.09356, 0.03235, 0.00799, 0.00475, 123669], '20694291': [0.03686, 0.00965, 0.00019, -0.00108, 489553], '1087289365544534016': [0.42763, 0.22874, 0.1783, 0.08632, 162364], '853507499181432832': [0.46275, 0.18675, 0.04821, 0.01574, 303351], '720786569792253952': [0.03202, 0.01028, 0.00255, 0.00159, 1220777], '1140820116': [0.06297, 0.03296, 0.01101, 0.0038, 590668], '191735009': [0.07531, 0.03556, 0.01032, 0.00497, 28099], '833038240122814464': [0.42101, 0.14644, 0.04392, 0.02005, 293800], '1069636036291112960': [0.43806, 0.15863, 0.0367, 0.0172, 45598], '4923859565': [0.06413, 0.04796, 0.02565, 0.01004, 47391], '304808873': [0.09338, 0.04715, 0.01409, 0.00605, 1012882], '471741741': [0.06481, 0.03547, 0.01039, 0.00568, 42707382], '48699128': [0.08464, 0.06933, 0.03943, 0.02736, 18401], '34427421': [0.03433, 0.01107, 0.00241, 0.00132, 603840], '568124708': [0.03655, 0.0218, 0.00926, 0.00336, 1101177], '154052153': [0.05366, 0.01805, 0.00343, 0.00313, 13765], '169491614': [0.0488, 0.01274, 0.00076, 0.00067, 11928], '1093246992': [0.09422, 0.07357, 0.03139, 0.01017, 656513], '1165728211': [0.09691, 0.03631, 0.00573, 0.00262, 19491], '17813487': [0.02265, 0.01048, 0.00041, 0.00026, 9116931], '742177082': [0.06279, 0.02647, 0.00756, 0.00419, 15590], '717618747423350785': [0.13179, 0.08646, 0.04286, 0.01762, 72094], '386373619': [0.065, 0.0423, 0.02796, 0.00491, 394191], '245956466': [0.04942, 0.0215, 0.00833, 0.00289, 9515657], '398212417': [0.15739, 0.06852, 0.02281, 0.01173, 2647441], '26981455': [0.46935, 0.20384, 0.0568, 0.02989, 144168], '316291124': [0.07604, 0.03283, 0.00722, 0.0036, 271770], '108437200': [0.11895, 0.04725, 0.00968, 0.00459, 203637], '4613834835': [0.09642, 0.0515, 0.01011, 0.00147, 371287], '160355694': [0.28861, 0.22958, 0.11613, 0.10044, 304769], '47863038': [246319.0, 0.02723, 0.00389, 0.00202, 246320], '46247180': [0.02053, 0.0073, 0.00154, 0.00088, 11327714], '718180293962776577': [0.23498, 0.09084, 0.0153, 0.0093, 81406], '108247668': [0.08014, 0.05325, 0.01712, 0.00807, 3082472], '2349940394': [0.09324, 0.05024, 0.01843, 0.00754, 423993], '269780552': [0.04251, 0.0316, 0.00852, 0.00376, 1556434], '48359021': [0.0538, 0.02592, 0.00853, 0.00421, 407282], '295218901': [0.96665, 0.61965, 0.32405, 0.08117, 2030834], '23951122': [0.18526, 0.07655, 0.01958, 0.00858, 16455], '2842600971': [0.04595, 0.03126, 0.01454, 0.01148, 611691], '65444625': [0.04276, 0.01468, 0.00419, 0.00242, 693172], '867617849208037377': [0.83386, 0.54896, 0.15133, 0.0559, 431939], '3406899801': [0.11371, 0.06009, 0.02291, 0.00754, 491376], '2273064468': [0.13954, 0.09025, 0.03866, 0.01685, 47307], '74015550': [0.01901, 0.00595, 0.00088, 0.00071, 1071198], '34543935': [0.57166, 0.04696, 0.00242, 0.00013, 226717], '985060027311575040': [0.23286, 0.08295, 0.02549, 0.01125, 10340], '1421528167': [0.2973, 0.22286, 0.11563, 0.05646, 213628], '37322526': [0.09937, 0.07679, 0.06764, 0.06717, 1882899], '151154260': [0.01876, 0.00634, 0.00108, 0.00083, 1241604], '23566038': [0.03156, 0.02053, 0.003, 0.00221, 212002], '956237994792538113': [0.14615, 0.05651, 0.01415, 0.00963, 137428], '36928426': [0.01015, 0.00525, 0.00064, -0.00066, 1038854], '1252301904': [0.18804, 0.07554, 0.02587, 0.00981, 375519], '32339364': [0.08508, 0.0362, 0.00926, 0.00454, 4501711], '816213211150188544': [480103.0, 0.23262, 0.08931, 0.01375, 480104], '2791298838': [0.13408, 0.0626, 0.01193, 0.0061, 26719], '715847424': [0.03389, 0.0143, 0.00343, 0.00152, 318423], '21931363': [0.01637, 0.00599, 0.00178, 0.00102, 5991405], '394011241': [0.15371, 0.02768, 0.00992, 0.00402, 254197], '272613386': [0.03655, 0.01303, 0.0037, 0.00113, 8502516], '413428207': [0.13348, 0.06575, 0.02372, 0.01286, 1485259], '17192256': [44348.0, 0.13651, 0.04174, 0.02043, 44349], '3026140439': [0.06738, 0.02477, 0.00696, 0.00365, 714430], '80298155': [0.07852, 0.04437, 0.01384, 0.00948, 33115], '270898732': [2.10657, 0.49981, 0.27375, 0.05734, 11544], '2260345147': [0.06416, 0.04316, 0.02563, 0.01255, 3915339], '310709509': [0.10133, 0.03555, 0.00721, 0.004, 2937697], '2680430490': [0.78323, 0.16617, 0.03712, 0.01386, 23412], '59063969': [0.00836, 0.00488, 0.00176, 0.00075, 30653], '1004014036927709185': [0.15188, 0.06309, 0.02503, 0.01318, 3345158], '1929596791': [0.18897, 0.10698, 0.03212, 0.00686, 3173442], '452653869': [0.17309, 0.07507, 0.01028, 0.00313, 1614429], '1330718940': [0.75556, 0.36571, 0.17479, 0.11038, 878450], '574032254': [1.11269, 0.59315, 0.1533, 0.05529, 2746176], '377976301': [0.06946, 0.0166, 0.00637, 0.00512, 52425], '79942437': [0.01625, 0.00601, 0.00146, 0.00073, 1808450], '884397469': [0.03491, 0.01572, 0.00327, 0.00164, 119184], '1490999844': [0.02048, 0.01308, 0.00591, 0.00231, 113435], '3028266616': [0.06068, 0.0247, 0.0046, 0.00297, 88903], '3382415499': [0.03577, 0.01272, 0.00504, 0.00296, 668318], '1168167671151628290': [0.24783, 0.08543, 0.02408, 0.01279, 711305], '164263382': [0.12333, 0.04958, 0.00476, 0.00383, 632598], '107162175': [0.05598, 0.02152, 0.01165, 0.00493, 38310], '278519480': [0.0089, 0.00217, 0.00023, 0.0002, 2958918], '1087731954756304896': [0.35087, 0.12573, 0.02905, 0.013, 118332], '498663728': [0.01666, 0.00857, 0.00019, -9e-05, 102943], '1436066864': [0.11897, 0.10706, 0.04151, 0.01278, 31611], '135116718': [0.04204, 0.02043, 0.00234, 0.00221, 23128], '25695032': [0.1378, 0.04928, 0.00753, 0.00427, 349306], '112764971': [0.0482, 0.02332, 0.00554, 0.00334, 16808721], '121087567': [0.04054, 0.01704, 0.00688, 0.00568, 100158], '2174419418': [0.10922, 0.04737, 0.01328, 0.00726, 370977], '3063032281': [0.03441, 0.02772, -0.001, -0.00173, 7199344], '2463676838': [0.03226, 0.02042, 0.0056, 0.00207, 1878503], '69261908': [0.0254, 0.00957, 0.00116, 0.00039, 198139], '723053842267398144': [0.26984, 0.12025, 0.02712, 0.01101, 77416], '19583545': [0.07956, 0.03522, 0.01144, 0.00553, 17372141], '2244953047': [3458733.0, 0.06275, 0.01577, 0.0066, 3458734], '567192377': [0.30585, 0.15636, 0.08485, 0.07809, 62856], '719870195695816704': [0.38632, 0.073, 0.04414, 0.03792, 130164], '2853870821': [0.19689, 0.11568, 0.02403, 0.01303, 44868], '258973851': [0.64635, 0.46766, 0.1438, 0.10409, 132643], '318385857': [0.02762, 0.00987, 0.00287, 6e-05, 1917821], '233033221': [0.24168, 0.12798, 0.05521, 0.03521, 668266], '64565898': [0.16644, 0.08567, 0.02973, 0.0196, 4954475], '41360559': [0.03885, 0.01422, 0.00367, 0.00166, 1784333], '991267091025051648': [1.00597, 0.335, 0.04463, 0.01378, 124236], '276799677': [0.1885, 0.13853, 0.0421, 0.01568, 2512822], '526951671': [0.02237, 0.0149, 0.0112, 0.00395, 2136163], '304982427': [0.06426, 0.0245, 0.00976, 0.00699, 904253], '30208416': [0.06982, 0.03736, 0.01196, 0.00369, 72171], '340566737': [0.07529, 0.03414, 0.00551, 0.00469, 14782], '229375729': [0.10016, 0.04474, 0.00807, 0.00404, 22606], '321862703': [0.0649, 0.03092, 0.00468, 0.00148, 22970], '748284237359513600': [0.4923, 0.41761, 0.03402, 0.01801, 42911], '1115518243': [0.09231, 0.01998, 0.00464, 0.00277, 150524], '111006961': [0.08804, 0.03443, 0.01034, 0.00555, 336516], '34611886': [0.17468, 0.07009, 0.03159, 0.01178, 53785], '1449069661': [0.09629, 0.04424, 0.00713, 0.00324, 286485], '18839785': [0.07086, 0.0384, 0.01261, 0.00636, 68968583], '163082968': [0.01382, 0.00658, 0.00109, 0.00049, 4903129], '38154763': [0.07384, 0.03181, 0.00852, 0.00434, 484639], '168683422': [0.15985, 0.07559, 0.02698, 0.01147, 21379647], '1016198241417822208': [0.09347, 0.03184, 0.00835, 0.00295, 7228395], '222360133': [0.31813, 0.30849, 0.09947, 0.04531, 137317], '811229675758505984': [0.15374, 0.06118, 0.01785, 0.00628, 1278125], '16712223': [0.0678, 0.03693, 0.01064, 0.00498, 1694182], '96951800': [0.05706, 0.02742, 0.00684, 0.00382, 36900170], '250371316': [0.07408, 0.05851, 0.02394, 0.0064, 1130013], '14470793': [0.09195, 0.04263, 0.00685, 0.00468, 267700], '213278292': [0.08288, 0.04478, 0.01336, 0.00625, 1862749], '1120633726478823425': [0.15738, 0.06985, 0.01319, 0.00579, 291759], '461005402': [0.02777, 0.01233, 0.00551, 0.00256, 152512], '833874495618293760': [0.0487, 0.00534, 0.00137, 0.00099, 40309], '390901101': [0.12757, 0.04767, 0.01097, 0.00572, 101955], '246181702': [0.14516, 0.06586, 0.0205, 0.00835, 4885139], '368430511': [0.08173, 0.04407, 0.01248, 0.00648, 478400], '1274553054': [0.04663, 0.01994, 0.00578, 0.00252, 1184892], '276611757': [0.02973, 0.01375, 0.00157, 0.00113, 1280014], '497329859': [0.20895, 0.06302, 0.00533, 0.00265, 178151], '1976143068': [0.0797, 0.03404, 0.00831, 0.0042, 7029010], '976995099170062338': [0.09318, 0.0527, 0.01371, 0.00972, 263537], '3353383450': [0.12073, 0.0588, 0.01639, 0.00879, 333471], '3048930572': [0.06661, 0.04288, 0.0147, 0.0076, 55551], '339620073': [0.07389, 0.03621, 0.00535, 0.00223, 3993634], '1462725421': [0.08478, 0.043, 0.01639, 0.00692, 1403617], '1429480158': [0.19297, 0.05364, 0.01439, 0.00383, 8299675], '9409752': [0.09213, 0.03572, 0.00693, 0.00344, 536546], '1005882189752557573': [0.15803, 0.09685, 0.03111, 0.01427, 32617], '3341967442': [0.01015, 0.00175, 0.00158, 0.00036, 678862], '453507741': [0.04328, 0.0212, 0.00633, 0.00278, 6137018], '485801262': [0.07795, 0.0411, 0.01415, 0.00675, 967060], '139876086': [0.05227, 0.02337, 0.00147, 0.00078, 7781504], '2870463431': [0.08711, 0.04049, 0.01125, 0.00517, 681557], '66200939': [0.06154, 0.03041, 0.01051, 0.00333, 238361], '748710513161404417': [0.18955, 0.1041, 0.03502, 0.0116, 120164], '176337215': [0.0335, 0.01757, 0.00597, 0.00389, 436188], '3171712086': [0.1161, 0.06955, 0.02691, 0.01217, 18986424], '586291040': [0.06199, 0.0198, 0.00497, 0.00219, 1154071], '181338564': [0.04186, 0.01658, 0.00741, 0.00291, 284728], '189868631': [0.06522, 0.02493, 0.00821, 0.00414, 1466787], '961930212656275456': [0.32005, 0.18348, 0.03747, 0.01847, 793704], '300811189': [0.07563, 0.02717, 0.00156, 0.00108, 23096], '25985333': [0.06838, 0.03411, 0.00822, 0.00486, 2928342], '72313888': [0.01809, 0.00823, 0.00193, 0.00149, 684693], '1252764865': [0.01534, 0.00942, 0.00252, 0.00137, 3916734], '214413743': [0.10855, 0.07636, 0.01754, 0.00893, 2237608], '333935142': [0.07245, 0.03559, 0.01134, 0.00502, 3895066], '68034431': [0.0343, 0.01447, 0.00512, 0.00122, 17868396], '58469274': [0.26308, 0.06435, 0.01676, 0.00693, 32268], '956092082728456192': [0.08351, 0.03268, 0.01572, 0.00998, 95882], '88716854': [0.06805, 0.02932, 0.01022, 0.0053, 799627], '568981074': [0.03424, 0.01258, 0.00045, 0.00044, 104910], '285577145': [0.43991, 0.16941, 0.06165, 0.03228, 255298], '141627220': [0.09671, 0.08807, 0.01069, 0.0048, 959779], '1597467145': [0.09219, 0.03914, 0.00988, 0.00511, 12984], '4502695341': [0.06347, 0.02117, 0.00406, 0.00096, 243141], '143456920': [0.03051, 0.01278, 0.00522, 0.00309, 223627], '885835554776121344': [0.13388, 0.06544, 0.03091, 0.01193, 381071], '69205157': [0.03643, 0.02738, 0.01364, 0.00817, 209500], '101895924': [0.00736, 0.00471, 0.0029, 0.00154, 1621811], '2317161228': [1183442.0, 0.08386, 0.02514, 0.01265, 1183443], '306642753': [0.11433, 0.07711, 0.02639, 0.01197, 702053], '39215566': [0.03751, 0.00966, 0.00316, -0.00034, 151810], '1287142711': [0.13911, 0.1154, 0.01839, 0.00554, 146557], '14688774': [0.1947, 0.08245, 0.02717, 0.00992, 876910], '169557169': [0.15914, 0.1134, 0.04354, 0.0214, 751376], '892792496': [0.04343, 0.01781, 0.00481, 0.00289, 489840], '1063745455': [0.24953, 0.09186, 0.02371, 0.01297, 23190], '1059327333532749824': [0.24997, 0.10658, 0.03534, 0.01727, 8150488], '316577087': [0.055, 0.0263, 0.00734, 0.00367, 2760674], '60327493': [0.01249, 0.00541, 0.00117, 0.00058, 13488861], '11525172': [0.05349, 0.01592, 0.00366, 0.00115, 8219865], '85270883': [0.0324, 0.01198, 0.0033, 0.00172, 3558943], '247217109': [0.08364, 0.03803, 0.00945, 0.00503, 1957900], '1972216999': [0.0194, 0.0102, 0.00476, 0.00123, 2865259], '1964571728': [0.06482, 0.03931, 0.01484, 0.00551, 12608436], '94438031': [0.05581, 0.0221, 0.00681, 0.00411, 384955], '2204673438': [0.16994, 0.0632, 0.01374, 0.00643, 51193], '9633802': [0.0388, 0.02347, 0.00376, 0.00146, 7489930], '295026890': [0.55114, 0.35223, 0.13395, 0.04882, 7783390], '108870336': [0.14661, 0.05921, 0.01599, 0.00861, 126170], '492852412': [0.02928, 0.01167, 0.00374, 0.00237, 380762], '558797310': [0.069, 0.03257, 0.00967, 0.00483, 25674425], '216289357': [0.05993, 0.03107, 0.01319, 0.00562, 6758218], '1082167187006242817': [0.1598, 0.12122, 0.06497, 0.02007, 146349], '63050007': [0.05263, 0.02351, 0.0048, 0.00199, 58424], '828410488509734912': [0.14221, 0.0479, 0.01599, 0.00804, 970641], '63098754': [0.1845, 0.08553, 0.03344, 0.01642, 2930132], '769148988511641601': [0.1316, 0.07261, 0.02674, 0.00656, 428623], '76178329': [0.10431, 0.07841, 0.04198, 0.03849, 998277], '123528331': [0.06271, 0.03196, 0.0045, 0.00209, 247516], '54879753': [0.01555, 0.00963, 0.0036, 0.00137, 95163], '570269160': [0.15667, 0.08379, 0.02811, 0.01111, 264177], '77596200': [0.0803, 0.02066, 0.00339, 0.00187, 7254103], '1349149096909668363': [11706199.0, 0.28007, 0.07788, 0.03125, 11706200], '297213631': [0.02374, 0.00921, 0.00183, 0.0011, 676865], '608112259': [0.18926, 0.09079, 0.02282, 0.00957, 21421], '507610089': [0.17432, 0.05775, 0.01568, 0.00864, 76580], '1307725646': [0.1565, 0.08226, 0.03957, 0.01878, 379393], '1952775169': [0.08833, 0.03778, 0.01038, 0.00626, 38269], '314547227': [0.00941, 0.00666, 0.00488, 0.00049, 4921488], '941002573': [0.13052, 0.03408, 0.00467, 0.00311, 16778], '314409266': [0.08204, 0.02812, 0.00683, 0.00323, 25192], '558739638': [0.03702, 0.02065, 0.00912, 0.00246, 245373], '211395583': [0.14003, 0.09493, 0.05288, 0.00899, 2143069], '1648317175': [0.07578, 0.02023, 0.00423, 0.00125, 24915], '2535581429': [0.08294, 0.03712, 0.01352, 0.00626, 523359], '511872699': [0.04651, 0.02924, 0.01682, 0.0144, 115043], '97675435': [0.03745, 0.01143, 0.00092, 0.00069, 151910], '115639376': [0.06365, 0.03323, 0.01603, 0.00538, 5784284], '256116885': [0.07199, 0.04804, 0.0187, 0.00286, 1362500], '407054921': [0.02542, 0.01307, 0.00425, 0.00213, 10390496], '779821454': [0.26374, 0.09126, 0.02907, 0.01484, 270074], '1730598050': [0.19382, 0.06643, 0.02446, 0.01413, 21527], '470307418': [0.01032, 0.00369, 0.00065, -8e-05, 15299020], '25357817': [0.08349, 0.06006, 0.03366, 0.01817, 106778], '2341884084': [0.12687, 0.03673, 0.00737, 0.00407, 1357283], '1446574225': [0.15266, 0.06734, 0.02588, 0.0138, 332247], '298031288': [0.03555, 0.01301, 0.00198, 0.001, 48117], '32246010': [0.05708, 0.02658, 0.0077, 0.00741, 81300], '1359603444': [0.23394, 0.14993, 0.06178, 0.00189, 30250], '96083073': [0.01575, 0.01573, 0.00739, 0.00235, 5701373], '105499378': [0.22997, 0.09018, 0.01204, 0.00643, 934649], '235827932': [0.06112, 0.04178, 0.0194, 0.00636, 82135], '60370151': [0.04933, 0.03518, 0.01464, 0.00234, 981732], '360099220': [0.01755, 0.0069, 0.00164, 0.0004, 1428058], '517559170': [0.12153, 0.05278, 0.01929, 0.01011, 1068068], '370112160': [0.13507, 0.06488, 0.00964, 0.00485, 528701], '941191189': [0.1212, 0.05256, 0.01644, 0.00859, 92424], '233644048': [0.06054, 0.0323, 0.01055, 0.00481, 451868], '112057739': [0.09533, 0.03271, 0.01084, 0.00597, 5327958], '2946300174': [0.1432, 0.09571, 0.04921, 0.02564, 264723], '3437747915': [0.20351, 0.14862, 0.06686, 0.03743, 168975], '859280289654341636': [0.23688, 0.10504, 0.02837, 0.0123, 2208820], '28631535': [0.02012, 0.01012, 0.00418, 0.00231, 9394972], '23573083': [0.02261, 0.00915, 0.00289, 0.001, 20530930], '717632257': [0.06183, 0.02598, 0.0075, 0.00371, 456945], '1264208004': [0.09799, 0.04155, 0.01699, 0.00768, 7246207], '536329828': [0.05042, 0.015, 0.00083, 0.00072, 225951], '1448212002': [0.11409, 0.04414, 0.00746, 0.00289, 103935], '197493438': [0.06109, 0.02382, 0.00591, 0.00347, 1667147], '78335735': [0.24258, 0.13725, 0.03484, 0.01504, 3416670], '58244743': [0.02448, 0.01107, 0.00072, 0.00029, 444007], '413756746': [0.09319, 0.03874, 0.01282, 0.00666, 862028], '1071390440': [0.07168, 0.03249, 0.01359, 0.00725, 30440], '3310241819': [0.16994, 0.07137, 0.02775, 0.01449, 289283], '105137891': [0.10142, 0.04389, 0.01105, 0.00533, 2399982], '32660559': [0.07273, 0.02822, 0.00671, 0.00334, 8594313], '41325048': [0.12419, 0.0478, 0.01326, 0.00742, 1861905], '410675207': [0.05036, 0.02325, 0.00791, 0.00429, 603947], '18055279': [0.07225, 0.03464, 0.00732, 0.00395, 1780768], '1356593833': [0.07057, 0.02325, 0.00959, 0.00565, 385257], '188870982': [0.02271, 0.00811, 0.00162, 0.00117, 417237], '119174866': [0.0282, 0.01738, 0.00629, 0.00465, 3277972], '1584684470': [0.02435, 0.0167, 0.0082, 0.00483, 183961], '25979455': [0.07058, 0.03682, 0.00787, 0.0057, 2425270], '1251590074053115909': [0.17397, 0.05873, 0.02036, 0.01326, 98024], '1375451460': [0.10793, 0.07803, 0.02368, 0.00716, 2707254], '380180989': [0.07936, 0.02938, 0.00349, 0.00156, 159434], '34935938': [0.02105, 0.00808, 0.00186, 0.00173, 1276648], '37665086': [0.08963, 0.02577, 0.00571, 0.00331, 267686], '2423100775': [0.27891, 0.20649, 0.00945, 0.00477, 129421], '143839873': [0.04607, 0.03224, 0.01531, 0.00903, 167368], '726748106': [0.03048, 0.01088, 0.00163, 0.00076, 1909010], '950313240': [0.06716, 0.03223, 0.007, 0.00322, 663169], '82115864': [0.08277, 0.03613, 0.008, 0.00324, 17635], '557295896': [0.09654, 0.04074, 0.01533, 0.0078, 167804], '3084419829': [0.48547, 0.19333, 0.06178, 0.02826, 23252], '11348282': [0.10134, 0.04372, 0.00956, 0.00511, 46941495], '109556877': [0.05574, 0.0224, 0.00691, 0.0029, 2133368], '781613052': [0.32048, 0.1616, 0.01924, 0.00755, 234572], '248692867': [0.03764, 0.02052, 0.00731, 0.00495, 3102795], '28191953': [0.03257, 0.01484, 0.00293, 0.00173, 1047854], '278235826': [0.01025, 0.00458, 0.00148, 0.00033, 15406725], '16033530': [0.25395, 0.10646, 0.02641, 0.01412, 67657], '14333756': [0.06185, 0.0268, 0.00727, 0.00344, 1900402], '20087934': [0.06547, 0.03377, 0.01018, 0.00518, 1692782], '3222211609': [0.10963, 0.04552, 0.01357, 0.00615, 1412535], '3277275168': [0.06401, 0.02839, 0.00238, 0.00174, 285959], '54296198': [0.02701, 0.01008, 0.00308, 0.00135, 2499227], '2330223926': [0.10044, 0.04357, 0.0169, 0.00595, 1640656], '1619325942': [0.15458, 0.07267, 0.03519, 0.01505, 203808], '15872418': [0.03271, 0.01471, 0.00486, 0.00226, 7436744], '161734463': [0.07166, 0.03898, 0.01821, 0.00688, 3755921], '471130203': [0.04159, 0.01602, 0.00318, 0.00176, 88272], '565752397': [0.03638, 0.01864, 0.00507, 0.00223, 1983743], '806373975894933504': [0.05063, 0.01352, 0.00431, 0.0033, 17017], '1351188841558913025': [2907005.0, 1.22611, 0.14093, 0.05392, 2907006], '186712788': [0.01671, 0.00849, 0.00059, 0.00011, 14104420], '153433497': [0.03504, 0.01637, 0.00187, 0.00102, 7571646], '3008454587': [0.1602, 0.06113, 0.0139, 0.00675, 19981], '75711015': [0.08456, 0.02369, 0.00193, 0.00095, 208966], '1430793200': [0.11344, 0.06346, 0.01981, 0.00945, 3285637], '103591569': [0.09566, 0.05646, 0.03189, 0.01271, 3855914], '1312452088547487744': [0.83535, 0.25286, 0.07559, 0.0338, 140399], '373257377': [0.10623, 0.09023, 0.05775, 0.00631, 576686], '1222273871488917504': [0.21751, 0.07709, 0.02137, 0.00843, 159511], '3923179469': [0.07026, 0.02171, 0.00666, 0.00294, 573191], '2793952751': [0.12567, 0.08186, 0.04932, 0.0221, 2018577], '1072915220': [0.02554, 0.01644, 0.00582, 0.00264, 9138452], '300760023': [0.06905, 0.0272, 0.00302, 0.00274, 14274], '255142088': [0.05306, 0.03617, 0.01056, 0.00858, 780002], '18186487': [0.11002, 0.04021, 0.00828, 0.00412, 342471], '46376581': [0.06916, 0.03488, 0.01051, 0.00415, 826140], '13197402': [0.13781, 0.10983, 0.06605, 0.02299, 317728], '1004204621445791744': [0.40201, 0.38535, 0.19597, 0.02154, 120294], '1143472657': [0.15439, 0.10888, 0.02928, 0.01346, 855424], '207120515': [0.07854, 0.05004, 0.02553, 0.00019, 575785], '30054530': [0.03065, 0.01437, 0.00279, 0.00171, 1071119], '440072927': [0.11159, 0.06279, 0.02416, 0.00662, 557548], '76278864': [0.15727, 0.06119, 0.01836, 0.00885, 399233], '742284510507761664': [0.15153, 0.0619, 0.01651, 0.00748, 13603], '2559482497': [0.12343, 0.03712, 0.00945, 0.00498, 115700], '816646997356777472': [0.36146, 0.15305, 0.03785, 0.0151, 121035], '71201743': [0.07192, 0.0356, 0.01018, 0.00498, 42506625], '311617637': [1690295.0, 0.07174, 0.02209, 0.01063, 1690296], '1898968398': [49097.0, 0.25359, 0.1885, 0.18291, 49098], '122465198': [0.08228, 0.03976, 0.00987, 0.00473, 69872], '214942749': [0.01388, 0.00859, 0.00216, 0.00113, 6257242], '51154192': [0.02579, 0.0092, 0.00194, 0.00117, 902162], '873755605': [0.07076, 0.04331, 0.0175, 0.00641, 8644822], '66329707': [0.02503, 0.0119, 0.00385, 0.0027, 462303], '870226459': [0.01597, 0.0064, 0.00138, 0.00086, 89309], '270804522': [0.09999, 0.05055, 0.01223, 0.00621, 30132], '328894456': [0.06479, 0.01974, 0.00928, 0.00501, 585974], '817736921027854336': [0.2116, 0.08598, 0.02545, 0.01203, 515187], '2744388988': [0.10994, 0.04232, 0.01676, 0.00787, 301805], '425871040': [0.07483, 0.03238, 0.00866, 0.00579, 13091980], '3240385810': [0.08716, 0.0347, 0.01041, 0.00629, 854633], '171299971': [0.05542, 0.02862, 0.00908, 0.00445, 1842777], '1164937093656862720': [0.17584, 0.11219, 0.00724, -0.00239, 1614142], '260539856': [0.14797, 0.06271, 0.02103, 0.00632, 815813], '1030316358': [0.03365, 0.02346, 0.01176, 0.00382, 165895], '38797799': [0.03736, 0.01467, 0.00387, 0.00272, 4889710], '57027096': [0.22803, 0.11396, 0.0448, 0.01605, 1967726], '237189703': [0.03881, 0.01922, 0.00506, 0.00295, 1873833], '81324580': [0.09554, 0.03353, 0.00757, 0.00454, 98599], '518004328': [0.13137, 0.06026, 0.00467, 0.00222, 924309], '177583133': [0.04658, 0.01997, 0.0068, 0.00385, 3693592], '53037279': [0.06941, 0.03553, 0.00861, 0.00401, 1554292], '594813423': [0.0367, 0.01884, 0.00925, 0.00215, 1667719], '2235729541': [10.43056, 1.7114, 0.33041, 0.09715, 1798805], '726133273': [0.11654, 0.04216, 0.01139, 0.00583, 413299], '102499820': [3.18908, 0.13549, 0.02758, 0.01088, 432275], '1878450026': [0.16231, 0.07814, 0.02198, 0.01016, 10625482], '71308607': [0.02874, 0.02218, 0.00429, 0.00216, 2713751], '1305893393835290624': [3049256.0, 0.35795, 0.11148, 0.04997, 3049257], '4660605282': [0.3334, 0.1374, 0.04194, 0.01908, 701728], '1055071081': [0.05996, 0.01559, 0.0021, 0.00181, 362408], '520547057': [0.07168, 0.04088, 0.02555, 0.00681, 1778112], '811505937911517184': [0.05814, 0.02433, 0.00727, 0.00333, 4664636], '3395146167': [0.11372, 0.07004, 0.02166, 0.01079, 1567255], '2602158362': [0.04843, 0.01402, 0.00614, 0.00402, 406723], '126676337': [0.04826, 0.012, 0.00189, 0.00131, 14279347], '61636241': [0.19368, 0.17481, 0.0281, 0.01631, 393673], '329162452': [5.87646, 0.17679, 0.02994, 0.01111, 328248], '1245289938': [0.09821, 0.04271, 0.01977, 0.01089, 418682], '20523263': [1.12732, 0.04534, 0.02187, 0.00866, 100369], '213539531': [0.04274, 0.01973, 0.00453, 0.00269, 6474041], '184688494': [0.13541, 0.04348, 0.01152, 0.00682, 979461], '274422326': [0.08533, 0.04904, 0.02349, 0.01466, 301795], '44670915': [0.05704, 0.02731, 0.00552, 0.00282, 9601730], '72881926': [0.06006, 0.0313, 0.01639, 0.0085, 183273], '497757669': [0.21471, 0.05241, 0.01074, 0.00563, 98518], '2162942852': [0.17811, 0.09159, 0.02955, 0.01281, 66482], '2260491445': [2.05906, 1.23847, 0.39111, 0.14075, 1957033], '158371427': [0.03044, 0.00974, 0.00121, 0.00045, 88423], '773280684173463552': [0.31523, 0.14921, 0.12507, 0.12358, 1079561], '319981198': [0.09059, 0.03849, 0.01211, 0.00535, 6300181], '1003757702': [0.06429, 0.01399, 0.00263, 0.0017, 515367], '122453931': [0.06012, 0.03249, 0.00881, 0.00367, 13838961], '1308877351359479808': [62596.0, 0.18096, 0.09856, 0.02286, 62597], '91053979': [0.00889, 0.0037, -0.0001, -0.00029, 3172091], '337119125': [0.22848, 0.16122, 0.07722, 0.04, 113169], '408245449': [0.05196, 0.02856, 0.01218, 0.00457, 664347], '70577513': [1768791.0, 0.92524, 0.09329, 0.03484, 1768792], '69934541': [0.10181, 0.03999, 0.02069, 0.01746, 67142], '1651594488': [0.16771, 0.0563, 0.02057, 0.01092, 709716], '34639484': [0.0495, 0.02233, 0.00749, 0.00479, 1358553], '3314485566': [0.35167, 0.13114, 0.03256, 0.01278, 3397589], '2977909943': [0.03146, 0.01831, 0.0031, 0.00094, 13902], '422861505': [0.10455, 0.04996, 0.01953, 0.00807, 5854031], '1960264604': [0.15479, 0.06396, 0.01416, 0.00709, 19046], '1729007461': [0.03192, 0.01557, 0.00493, 0.00312, 40188], '24291371': [0.1094, 0.04842, 0.0171, 0.00871, 4326560], '3246099789': [0.13241, 0.05605, 0.0084, 0.00377, 87115], '261923564': [0.33077, 0.22347, 0.07145, 0.03664, 167640], '2597010860': [0.6002, 0.25337, 0.06441, 0.02632, 2680064], '3919666337': [0.06201, 0.03507, 0.0109, 0.00532, 25143], '440662588': [0.37975, 0.20311, 0.05234, 0.02548, 2783945], '121482054': [0.03928, 0.01763, 0.00065, 0.00052, 228669], '186386857': [0.0298, 0.00958, 0.00214, 0.00149, 7828301], '558752511': [0.05338, 0.01745, 0.00533, 0.00244, 286733], '133272363': [0.08621, 0.02521, 0.00357, 0.00205, 27820], '2496702292': [0.08913, 0.04131, 0.01362, 0.00487, 221914], '1724982397': [0.13036, 0.06056, 0.00732, 0.00275, 12382], '537951506': [0.35633, 0.24793, 0.06792, 0.04226, 618945], '110935165': [0.38074, 0.11995, 0.03264, 0.01631, 16134], '538949379': [0.1223, 0.06118, 0.02141, 0.01038, 530355], '3280339711': [11.56729, 0.31103, 0.04944, 0.0185, 325455], '1428617245': [0.08174, 0.03426, 0.02306, 0.00297, 249034], '96941094': [0.05204, 0.01983, 0.00618, 0.00403, 389446], '3012716556': [0.15244, 0.05549, 0.01642, 0.00524, 547146], '59825777': [0.10306, 0.04916, 0.01603, 0.00775, 573440], '2721315419': [0.0234, 0.01122, 0.00294, 0.00109, 323989], '316053407': [0.01915, 0.00725, 0.00235, 0.00102, 7081710], '99914275': [0.18248, 0.04578, 0.01518, 0.00584, 25149], '301309392': [0.08343, 0.03312, 0.00962, 0.00515, 2760876]}

try:
  output = output
except:
  output = static_output

sorted_chg_6m = sorted([float(i[0]) for i in [i for i in output.values()]])
sorted_chg_3m = sorted([float(i[1]) for i in [i for i in output.values()]])
sorted_chg_1m = sorted([float(i[2]) for i in [i for i in output.values()]])
sorted_chg_2w = sorted([float(i[3]) for i in [i for i in output.values()]])

### Get top new follower % accounts

In [16]:
driver = webdriver.Chrome('chromedriver',options=chrome_options)
#Setting targets
target_list, target_value, tenor, top_x = set_var('2w',25) # set tenor and top # of profiles to view
target_count = 1
mydict = OrderedDict()

for i in target_list[::-1]:
  for key, value in output.items():
    if value[target_value] == i:# and int(value[0]+1) != value[4]: #To remove division by 0 followers data
      profile_ext = '/statistics/twitter/profiles/detail/' + key
      profile_name = get_profile_name(profile_ext)
      category = get_category(profile_ext)
      print(target_count,tenor+' chg = '+str(round(value[target_value]*100,2))+'%',profile_name, category, base+profile_ext, value)                                    
      mydict[profile_name] = {'URL' : base+profile_ext,
                              'Category' : category,
                              'Followers' : f"{value[4]:,d}",
                              '6m chg' : f"{value[0]:.0%}",#str(int(value[0]*100)) +'%',
                              '3m chg' : f"{value[1]:.0%}",
                              '1m chg' : f"{value[2]:.0%}",
                              '0.5m chg' : f"{value[3]:.0%}"}
  target_count+=1

1 2w chg = 45.4% @ONPE_oficial {'society/governmental', 'peru'} https://www.socialbakers.com/statistics/twitter/profiles/detail/255707941 [0.9937, 0.91543, 0.4596, 0.45404, 525490]
2 2w chg = 38.87% @intel_la {'global', 'brands/electronics'} https://www.socialbakers.com/statistics/twitter/profiles/detail/53459457 [0.44089, 0.41333, 0.38911, 0.38869, 91855]
3 2w chg = 29.03% @taoyuanairport {'taiwan', 'place/airport'} https://www.socialbakers.com/statistics/twitter/profiles/detail/3306946201 [0.97065, 0.90688, 0.29522, 0.29028, 19269]
4 2w chg = 26.56% @WePlay_Esports {'ukraine'} https://www.socialbakers.com/statistics/twitter/profiles/detail/949934119 [0.51783, 0.46346, 0.40693, 0.26562, 29575]
5 2w chg = 23.14% @TEAMEVGA {'brands/services', 'united-states', 'united-kingdom'} https://www.socialbakers.com/statistics/twitter/profiles/detail/26180582 [0.26966, 0.26344, 0.24535, 0.23136, 631722]
6 2w chg = 18.29% @carlitosrejala {'society/politics', 'paraguay'} https://www.socialbakers.com

In [17]:
mydict

OrderedDict([('@ONPE_oficial',
              {'0.5m chg': '45%',
               '1m chg': '46%',
               '3m chg': '92%',
               '6m chg': '99%',
               'Category': {'peru', 'society/governmental'},
               'Followers': '525,490',
               'URL': 'https://www.socialbakers.com/statistics/twitter/profiles/detail/255707941'}),
             ('@intel_la',
              {'0.5m chg': '39%',
               '1m chg': '39%',
               '3m chg': '41%',
               '6m chg': '44%',
               'Category': {'brands/electronics', 'global'},
               'Followers': '91,855',
               'URL': 'https://www.socialbakers.com/statistics/twitter/profiles/detail/53459457'}),
             ('@taoyuanairport',
              {'0.5m chg': '29%',
               '1m chg': '30%',
               '3m chg': '91%',
               '6m chg': '97%',
               'Category': {'place/airport', 'taiwan'},
               'Followers': '19,269',
               'URL': 'ht